In [1]:
!pip install numexpr
!pip install -U "setuptools<58" # setuptoolsのverが58以上だと、xfeatの依存パッケージml-metricsのinstallに失敗するため
!pip install optuna xfeat japanize-matplotlib
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 57.5.0 which is incompatible.
cvxpy 1.3.1 requires setuptools>65.5.1, but you have setuptools 57.5.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 65.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.3 MB/s eta 0:00:00
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=ac4549efcabc8d642c21d26b9d6f72d3c8c48c551c85ffb52b8003263d5087af
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-py3-none-any.whl size=7844 sha256=fc66966d0317768eee31b9f70eddb18cacad6f055bbdb135e1110fc631857f9b
  Stored in directory: /root/.cache/pip/wheels/45/c8/f0/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pathlib import Path
from glob import glob
path_start = Path("/content/drive/MyDrive/Nishika/Mansion_2023_spring")
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from xfeat import SelectCategorical, LambdaEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer
import category_encoders as ce
import scipy

def Read():
  train = pd.read_csv(path_start/"input/df_train.csv")
  test = pd.read_csv(path_start/"input/test.csv")
  df_ConCat = pd.concat([train, test])
  df_ConCat = df_ConCat.drop(["Unnamed: 0", "地域", "前面道路：方位", "前面道路：種類",	"前面道路：幅員（ｍ）",
                              "種類", "土地の形状", "間口", "延床面積（㎡）"], axis=1)
  return df_ConCat

Mounted at /content/drive


## 取引時期を年と月に分ける

In [ ]:
df = Read()
a1 = df["取引時点"].str.split(pat="年第", expand=True)
a2 = a1[1].str.split(pat="四半期", expand=True)
df_torihiki = pd.concat({"ID": df["ID"], "取引時期_年":a1[0].astype("int64"), "取引時期_四半期":a2[0].astype("int64")}, axis=1)
df_torihiki["取引時期_年四半期"] = df_torihiki["取引時期_年"] + (df_torihiki["取引時期_四半期"]/4 - 0.25)
df_torihiki["建築年"] = df["建築年"].replace(0, value=df["建築年"].median())
df_torihiki["取引-建築"] = df_torihiki["取引時期_年四半期"] - df_torihiki["建築年"]
df_torihiki["取引-建築"] = (df_torihiki["取引-建築"] - df_torihiki["取引-建築"].mean())/ df_torihiki["取引-建築"].std()
df_torihiki.head()

,ID,取引時期_年,取引時期_四半期,取引時期_年四半期,建築年,取引-建築
0,8067540,2007,3,2007.50,1996.0,-0.636647
1,8027265,2018,3,2018.50,1998.0,0.106899
2,8061526,2011,4,2011.75,2009.0,-1.359538
3,8086147,2013,1,2013.00,2007.0,-1.091036
4,8049498,2018,2,2018.25,2009.0,-0.822533


In [ ]:
df_torihiki.to_csv(path_start/"feature_value/df_取引時点.csv", index=False)
pd.read_csv(path_start/"feature_value/df_取引時点.csv")

,ID,取引時期_年,取引時期_四半期,取引時期_年四半期,建築年,取引-建築
0,8067540,2007,3,2007.50,1996.0,-0.636647
1,8027265,2018,3,2018.50,1998.0,0.106899
2,8061526,2011,4,2011.75,2009.0,-1.359538
3,8086147,2013,1,2013.00,2007.0,-1.091036
4,8049498,2018,2,2018.25,2009.0,-0.822533
...,...,...,...,...,...,...
784708,47006020,2022,2,2022.25,2017.0,-1.152998
784709,47006229,2022,3,2022.50,2016.0,-1.049728
784710,47006331,2022,3,2022.50,2018.0,-1.214960
784711,47006332,2022,2,2022.25,1999.0,0.334094


## 市町村を仕分ける

In [ ]:
df = Read()
df_city = df[df['市区町村名'].str.contains('市')]
df_no_city = df[df['市区町村名'].str.contains('市')==False]

a = df_city["市区町村名"].str.split("市", expand=True)
del a[2]
a = a.rename(columns={0: '市名', 1: "区名"})

a["区名"] = a["区名"].replace("", "名前なし")
df_CityName = pd.concat([df_city["ID"], a["市名"] + "市", a["区名"]], axis=1)

df_no_city_gun = df_no_city[df_no_city['市区町村名'].str.contains('郡')]
df_no_city_no_gun = df_no_city[df_no_city['市区町村名'].str.contains('郡')==False]

a2 = df_no_city_gun["市区町村名"].str.split("郡", expand=True)
#del a2[2]
a2 = a2.rename(columns={0: '郡名', 1: "町名"})

a2["町名"] = a2["町名"].replace("", "名前なし")
df_CityName2 = pd.concat([df_no_city_gun["ID"], a2["郡名"] + "郡", a2["町名"]], axis=1)

df_no_city_no_gun = df_no_city_no_gun.rename(columns={"市区町村名": '区名'})
df_CityName3 = pd.concat([df_no_city_no_gun["ID"], df_no_city_no_gun["区名"]], axis=1)

df_CityName_all = pd.concat([df_CityName, df_CityName2, df_CityName3])
df_CityName_all = df_CityName_all.fillna(value="名前なし")
df_CityName_all.head()

,ID,市名,区名,郡名,町名
0,8067540,取手市,名前なし,名前なし,名前なし
1,8027265,牛久市,名前なし,名前なし,名前なし
2,8061526,つくば市,名前なし,名前なし,名前なし
3,8086147,つくば市,名前なし,名前なし,名前なし
4,8049498,守谷市,名前なし,名前なし,名前なし


In [ ]:
df_CityName_all.to_csv(path_start/"feature_value/df_市町村.csv", index=False)
pd.read_csv(path_start/"feature_value/df_市町村.csv")

,ID,市名,区名,郡名,町名
0,8067540,取手市,名前なし,名前なし,名前なし
1,8027265,牛久市,名前なし,名前なし,名前なし
2,8061526,つくば市,名前なし,名前なし,名前なし
3,8086147,つくば市,名前なし,名前なし,名前なし
4,8049498,守谷市,名前なし,名前なし,名前なし
...,...,...,...,...,...
784708,13314689,名前なし,江戸川区,名前なし,名前なし
784709,13314692,名前なし,江戸川区,名前なし,名前なし
784710,13314695,名前なし,江戸川区,名前なし,名前なし
784711,13314699,名前なし,江戸川区,名前なし,名前なし


## 部屋を仕分ける

In [ ]:
df = Read()
df_room = df[["ID", "間取り"]]
df_room["間取り"] = df_room["間取り"].fillna("不明")

a = df_room["間取り"].str.split("＋", expand=True)
Room_Num = lambda x: re.sub(r"[^０-９]+", "", x)
Room_Num_not = lambda x: re.sub(r"[０-９]+", "", x)
a[2] = a[0].apply(Room_Num)
a[3] = a[0].apply(Room_Num_not)
a_new = a.rename(columns={0:"間取り", 1:"間取り_+",	2:"間取り_数字", 3:"間取り_種類"})

df_room = pd.concat([df_room["ID"], a_new[["間取り_+", "間取り_数字", "間取り_種類"]]], axis=1)

df_room_null = df_room[df_room["間取り_数字"]==""]
df_room_nonull = df_room[df_room["間取り_数字"]!=""]

df_room_null["間取り_数字"] = "０"
df_room2 = pd.concat([df_room_null, df_room_nonull])
df_room2["間取り_数字"] = df_room2["間取り_数字"].astype(int)
df_room2["間取り_+"] = df_room2["間取り_+"].fillna("不明")

<ipython-input-48-5d3da8bdf7f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_room["間取り"] = df_room["間取り"].fillna("不明")
<ipython-input-48-5d3da8bdf7f4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_room_null["間取り_数字"] = "０"


In [ ]:
df_room2.to_csv(path_start/"feature_value/df_間取り.csv", index=False)
pd.read_csv(path_start/"feature_value/df_間取り.csv")

,ID,間取り_+,間取り_数字,間取り_種類
0,8072280,不明,0,不明
1,8024229,不明,0,不明
2,8013693,不明,0,不明
3,8023527,不明,0,不明
4,8013810,不明,0,不明
...,...,...,...,...
784708,47003330,不明,3,ＬＤＫ
784709,47006229,不明,4,ＬＤＫ
784710,47006331,不明,2,ＬＤＫ
784711,47006332,不明,2,ＬＤＫ


## 建物の構造をonehot表現に

In [ ]:
df = Read()
List_deltaName = ['ＲＣ', '鉄骨造', 'ＳＲＣ', "不明", '木造', '軽量鉄骨造', 'ブロック造']
df_onehot = pd.DataFrame(df["ID"])
df["建物の構造"] = df["建物の構造"].fillna("不明")
a = df["建物の構造"].str.split("、", expand=True)

for x in List_deltaName:
  b = a==x
  b = b*1
  b = b[0]+b[1]+b[2]
  c = pd.DataFrame({"建物の構造_{}".format(x): b})
  df_onehot = pd.concat([df_onehot, c], axis=1)

df_onehot

,ID,建物の構造_ＲＣ,建物の構造_鉄骨造,建物の構造_ＳＲＣ,建物の構造_不明,建物の構造_木造,建物の構造_軽量鉄骨造,建物の構造_ブロック造
0,8067540,1,0,0,0,0,0,0
1,8027265,1,0,0,0,0,0,0
2,8061526,1,0,0,0,0,0,0
3,8086147,1,0,0,0,0,0,0
4,8049498,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
19296,47006020,1,0,0,0,0,0,0
19297,47006229,1,0,0,0,0,0,0
19298,47006331,1,0,0,0,0,0,0
19299,47006332,0,0,0,1,0,0,0


In [ ]:
df_onehot.to_csv(path_start/"feature_value/df_建物の構造.csv", index=False)
pd.read_csv(path_start/"feature_value/df_建物の構造.csv")

,ID,建物の構造_ＲＣ,建物の構造_鉄骨造,建物の構造_ＳＲＣ,建物の構造_不明,建物の構造_木造,建物の構造_軽量鉄骨造,建物の構造_ブロック造
0,8067540,1,0,0,0,0,0,0
1,8027265,1,0,0,0,0,0,0
2,8061526,1,0,0,0,0,0,0
3,8086147,1,0,0,0,0,0,0
4,8049498,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
784708,47006020,1,0,0,0,0,0,0
784709,47006229,1,0,0,0,0,0,0
784710,47006331,1,0,0,0,0,0,0
784711,47006332,0,0,0,1,0,0,0


## 用途を細分化

In [ ]:
df = Read()
df["用途"] = df["用途"].fillna("不明")
a = df["用途"].str.split("、", expand=True)
List_a = list(a[0].unique())
List_unique = []
x, y = a.shape

for i in range(y):
  List_a = list(a[i].unique())
  List_unique = List_unique or List_a

List_unique

['住宅', '不明', '店舗', 'その他', '事務所', '駐車場', '倉庫', '作業場', '工場']

In [ ]:
df_onehot = pd.DataFrame(df["ID"])

for x in List_unique:
  b = a==x
  b = b*1
  b = b[0]+b[1]+b[2]
  c = pd.DataFrame({"用途_{}".format(x): b})
  df_onehot = pd.concat([df_onehot, c], axis=1)

df_onehot.head()

,ID,用途_住宅,用途_不明,用途_店舗,用途_その他,用途_事務所,用途_駐車場,用途_倉庫,用途_作業場,用途_工場
0,8067540,1,0,0,0,0,0,0,0,0
1,8027265,1,0,0,0,0,0,0,0,0
2,8061526,1,0,0,0,0,0,0,0,0
3,8086147,1,0,0,0,0,0,0,0,0
4,8049498,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
19296,47006020,0,1,0,0,0,0,0,0,0
19297,47006229,0,1,0,0,0,0,0,0,0
19298,47006331,0,1,0,0,0,0,0,0,0
19299,47006332,1,0,0,0,0,0,0,0,0


In [ ]:
df_onehot.to_csv(path_start/"feature_value/df_用途.csv", index=False)
pd.read_csv(path_start/"feature_value/df_用途.csv")

,ID,用途_住宅,用途_不明,用途_店舗,用途_その他,用途_事務所,用途_駐車場,用途_倉庫,用途_作業場,用途_工場
0,8067540,1,0,0,0,0,0,0,0,0
1,8027265,1,0,0,0,0,0,0,0,0
2,8061526,1,0,0,0,0,0,0,0,0
3,8086147,1,0,0,0,0,0,0,0,0
4,8049498,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
784708,47006020,0,1,0,0,0,0,0,0,0
784709,47006229,0,1,0,0,0,0,0,0,0
784710,47006331,0,1,0,0,0,0,0,0,0
784711,47006332,1,0,0,0,0,0,0,0,0


## 用途と利用目的が一致してるか

In [ ]:
df = Read()
df_youto = pd.read_csv("/content/drive/MyDrive/Nishika/Mansion_2023_spring/feature_value/df_用途.csv")

df["今後の利用目的"] = df["今後の利用目的"].fillna("不明")
df_merge = pd.DataFrame(df["ID"])
for i in list(df["今後の利用目的"].unique()):
  a = pd.DataFrame(list(df_youto['用途_{}'.format(i)]==1))
  a *= 1
  b = pd.DataFrame(list(df['今後の利用目的']=="{}".format(i)))
  b *= 1
  c = a+b==2
  c *= 1
  df_merge["利用一致_{}".format(i)] = c

df_merge.head()

,ID,利用一致_不明,利用一致_住宅,利用一致_その他,利用一致_事務所,利用一致_店舗
0,8067540,0,0,0,0,0
1,8027265,0,1,0,0,0
2,8061526,0,0,0,0,0
3,8086147,0,0,0,0,0
4,8049498,0,0,0,0,0


In [ ]:
df_merge.to_csv(path_start/"feature_value/df_利用一致.csv", index=False)
pd.read_csv(path_start/"feature_value/df_利用一致.csv")

,ID,利用一致_不明,利用一致_住宅,利用一致_その他,利用一致_事務所,利用一致_店舗
0,8067540,0,0,0,0,0
1,8027265,0,1,0,0,0
2,8061526,0,0,0,0,0
3,8086147,0,0,0,0,0
4,8049498,0,0,0,0,0
...,...,...,...,...,...,...
784708,47006020,0,1,0,0,0
784709,47006229,0,0,0,0,0
784710,47006331,0,0,0,0,0
784711,47006332,0,0,0,0,0


## 敷地面積を求めてみよう

In [ ]:
df = Read()
df_area = pd.DataFrame(df["ID"])
df_area["敷地面積"] = df["面積（㎡）"]/(df["建ぺい率（％）"]/100)
df_area["延べ床面積"] = df_area["敷地面積"]*df["容積率（％）"]/100

df_area["敷地面積"] = (df_area["敷地面積"] - df_area["敷地面積"].mean())/ df_area["敷地面積"].std()
df_area["延べ床面積"] = (df_area["延べ床面積"] - df_area["延べ床面積"].mean())/ df_area["延べ床面積"].std()

df_area.to_csv(path_start/"feature_value/df_敷地面積.csv", index=False)
pd.read_csv(path_start/"feature_value/df_敷地面積.csv")

,ID,敷地面積,延べ床面積
0,8067540,0.597151,-0.046664
1,8027265,0.597151,-0.046664
2,8061526,0.919722,1.907805
3,8086147,1.703108,0.623440
4,8049498,0.643233,-0.018743
...,...,...,...
784708,47006020,0.366743,0.525716
784709,47006229,1.150129,0.288388
784710,47006331,0.781477,0.065020
784711,47006332,0.412825,-0.158348


## 取引の事情等を細分化

In [ ]:
df = Read()
df["取引の事情等"] = df["取引の事情等"].fillna("不明")
a = df["取引の事情等"].str.split("、", expand=True)
List_a = list(a[0].unique())
List_unique = []
x, y = a.shape

for i in range(y):
  List_a = list(a[i].unique())
  List_unique = List_unique or List_a

List_unique

['不明', '調停・競売等', '瑕疵有りの可能性', '関係者間取引', 'その他事情有り', '他の権利・負担付き']

In [ ]:
df_onehot = pd.DataFrame(df["ID"])

for x in List_unique:
  b = a==x
  b = b*1
  b = b[0]+b[1]
  c = pd.DataFrame({"取引の事情等_{}".format(x): b})
  df_onehot = pd.concat([df_onehot, c], axis=1)

df_onehot.head()

,ID,取引の事情等_不明,取引の事情等_調停・競売等,取引の事情等_瑕疵有りの可能性,取引の事情等_関係者間取引,取引の事情等_その他事情有り,取引の事情等_他の権利・負担付き
0,8067540,1,0,0,0,0,0
1,8027265,1,0,0,0,0,0
2,8061526,1,0,0,0,0,0
3,8086147,1,0,0,0,0,0
4,8049498,1,0,0,0,0,0


In [ ]:
df_onehot.to_csv(path_start/"feature_value/df_取引の事情等.csv", index=False)
pd.read_csv(path_start/"feature_value/df_取引の事情等.csv")

,ID,取引の事情等_不明,取引の事情等_調停・競売等,取引の事情等_瑕疵有りの可能性,取引の事情等_関係者間取引,取引の事情等_その他事情有り,取引の事情等_他の権利・負担付き
0,8067540,1,0,0,0,0,0
1,8027265,1,0,0,0,0,0
2,8061526,1,0,0,0,0,0
3,8086147,1,0,0,0,0,0
4,8049498,1,0,0,0,0,0
...,...,...,...,...,...,...,...
784708,47006020,1,0,0,0,0,0
784709,47006229,1,0,0,0,0,0
784710,47006331,1,0,0,0,0,0
784711,47006332,1,0,0,0,0,0


## 用途と今後をつなげる

In [ ]:
df = Read()
df_used = pd.DataFrame(df["ID"])
df["用途"] = df["用途"].fillna("不明")
df["今後の利用目的"] = df["今後の利用目的"].fillna("不明")
df_used["利用変異"] = df["用途"] + "から" + df["今後の利用目的"]
df_used.head()

,ID,利用変異
0,8067540,住宅から不明
1,8027265,住宅から住宅
2,8061526,住宅から不明
3,8086147,住宅から不明
4,8049498,不明から住宅


In [ ]:
df_used.to_csv(path_start/"feature_value/df_利用変異.csv", index=False)
pd.read_csv(path_start/"feature_value/df_利用変異.csv")

,ID,利用変異
0,8067540,住宅から不明
1,8027265,住宅から住宅
2,8061526,住宅から不明
3,8086147,住宅から不明
4,8049498,不明から住宅
...,...,...
784708,47006020,不明から店舗
784709,47006229,不明から住宅
784710,47006331,不明から住宅
784711,47006332,住宅から住宅


## チュートリアル

In [ ]:
df = Read()
df = df.drop(["種類", "土地の形状", "間口", "延床面積（㎡）"], axis=1)
df.head()

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,8067540,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,70,1996.0,ＲＣ,住宅,NaN,第１種住居地域,60.0,200.0,2007年第3四半期,未改装,NaN,6.977724
1,8027265,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,70,1998.0,ＲＣ,住宅,住宅,第２種中高層住居専用地域,60.0,200.0,2018年第3四半期,未改装,NaN,7.255273
2,8061526,8220,茨城県,つくば市,苅間,研究学園,4,３ＬＤＫ,105,2009.0,ＲＣ,住宅,NaN,商業地域,80.0,400.0,2011年第4四半期,未改装,NaN,7.556303
3,8086147,8220,茨城県,つくば市,並木,つくば,75,４ＬＤＫ,100,2007.0,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2013年第1四半期,未改装,NaN,7.380211
4,8049498,8224,茨城県,守谷市,ひがし野,守谷,8,４ＬＤＫ,95,2009.0,ＲＣ,NaN,住宅,近隣商業地域,80.0,200.0,2018年第2四半期,未改装,NaN,7.518514


In [ ]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)
df.head()

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,...,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc
0,8067540,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,70,1996.0,...,住宅,NaN,第１種住居地域,60.0,200.0,2007年第3四半期,未改装,NaN,6.977724,8
1,8027265,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,70,1998.0,...,住宅,住宅,第２種中高層住居専用地域,60.0,200.0,2018年第3四半期,未改装,NaN,7.255273,52
2,8061526,8220,茨城県,つくば市,苅間,研究学園,4,３ＬＤＫ,105,2009.0,...,住宅,NaN,商業地域,80.0,400.0,2011年第4四半期,未改装,NaN,7.556303,25
3,8086147,8220,茨城県,つくば市,並木,つくば,75,４ＬＤＫ,100,2007.0,...,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2013年第1四半期,未改装,NaN,7.380211,30
4,8049498,8224,茨城県,守谷市,ひがし野,守谷,8,４ＬＤＫ,95,2009.0,...,NaN,住宅,近隣商業地域,80.0,200.0,2018年第2四半期,未改装,NaN,7.518514,51


In [ ]:
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'
TARGET = '取引価格（総額）_log'

for i in set(df[time_col].values):
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic[50]

{'三重県': 7.099989772836439,
 '京都府': 7.155935848555253,
 '佐賀県': 7.009937258263509,
 '兵庫県': 7.157742870483196,
 '北海道': 6.965967793677859,
 '千葉県': 7.155825324964995,
 '和歌山県': 7.022690210815773,
 '埼玉県': 7.1697442713966355,
 '大分県': 6.911537826573079,
 '大阪府': 7.158352973821201,
 '奈良県': 7.056884610234688,
 '宮城県': 7.0395442538048805,
 '宮崎県': 6.958678653711235,
 '富山県': 7.0469200713870155,
 '山口県': 7.045342730349803,
 '山形県': 7.087990602814902,
 '山梨県': 6.848839888686708,
 '岐阜県': 7.078095913486453,
 '岡山県': 7.03809073492747,
 '岩手県': 6.925002679060599,
 '島根県': 7.165376214700312,
 '広島県': 7.093449021655064,
 '徳島県': 6.912051981629745,
 '愛媛県': 6.966433011596662,
 '愛知県': 7.107301969238601,
 '新潟県': 6.913835156235044,
 '東京都': 7.377929204641064,
 '栃木県': 6.950636958071011,
 '沖縄県': 7.160046604428336,
 '滋賀県': 7.16602695816839,
 '熊本県': 6.970182251843263,
 '石川県': 6.929363348877185,
 '神奈川県': 7.2581182672098175,
 '福井県': 6.979575815840195,
 '福岡県': 6.992264905042226,
 '福島県': 6.9743805791401785,
 '秋田県': 6.9413137471126

In [ ]:
def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df[group_col+'_te'] = df.apply(calc_te, axis=1)
df.head(2)

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,...,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te
0,8067540,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,70,1996.0,...,NaN,第１種住居地域,60.0,200.0,2007年第3四半期,未改装,NaN,6.977724,8,7.161165
1,8027265,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,70,1998.0,...,住宅,第２種中高層住居専用地域,60.0,200.0,2018年第3四半期,未改装,NaN,7.255273,52,7.073184


In [ ]:
num_df = SelectNumerical().fit_transform(df)
num_df.head(2)

,ID,市区町村コード,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te
0,8067540,8217,13,70,1996.0,60.0,200.0,6.977724,8,7.161165
1,8027265,8219,8,70,1998.0,60.0,200.0,7.255273,52,7.073184


In [ ]:
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)
le_df.head()

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,取引時点,改装,取引の事情等
0,0,0,0,0,0,0,0,-1,0,0,0,-1
1,0,1,1,1,0,0,0,0,1,1,0,-1
2,0,2,2,2,0,0,0,-1,2,2,0,-1
3,0,2,3,3,1,0,0,-1,3,3,0,-1
4,0,3,4,4,1,0,-1,0,4,4,0,-1


In [ ]:
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=["面積（㎡）", "容積率（％）"],
            drop_origin=True,
            operator="*",
            r=2,
        ),
    ]
)

num_comb_df = encoder.fit_transform(df)/100
num_comb_df.head()

,面積（㎡）容積率（％）_combi
0,140.0
1,140.0
2,420.0
3,200.0
4,190.0


In [ ]:
agg_dfs = []

def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(df,
                        group_key=group_col,
                        group_values=['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）'],
                        agg_methods=['count', 'mean', 'min', 'max'],
                        )

    return agg_df[agg_cols]

group_col = '市区町村名'
agg_dfs.append(get_agg_df(df, group_col))
agg_dfs[0].head()

,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
0,690,690,674,674,12.040580,63.775362,62.477745,217.759644,1,15,40.0,80.0,75,970,80.0,500.0
1,344,344,340,340,6.308140,72.034884,72.823529,320.294118,-1,60,60.0,200.0,23,115,80.0,400.0
2,739,739,688,688,22.109608,85.548038,68.313953,261.991279,-1,55,40.0,60.0,75,390,80.0,400.0
3,739,739,688,688,22.109608,85.548038,68.313953,261.991279,-1,55,40.0,60.0,75,390,80.0,400.0
4,156,156,155,155,7.339744,76.987179,73.677419,239.354839,1,55,60.0,100.0,19,105,80.0,500.0


In [ ]:
df_concat = pd.concat([df["ID"], pd.DataFrame(agg_dfs[0])], axis=1)

In [ ]:
df_concat.to_csv(path_start/"feature_value/df_チュートリアル.csv", index=False)
pd.read_csv(path_start/"feature_value/df_チュートリアル.csv")

,ID,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
0,8067540,690,690,674,674,12.040580,63.775362,62.477745,217.759644,1,15,40.0,80.0,75,970,80.0,500.0
1,8027265,344,344,340,340,6.308140,72.034884,72.823529,320.294118,-1,60,60.0,200.0,23,115,80.0,400.0
2,8061526,739,739,688,688,22.109608,85.548038,68.313953,261.991279,-1,55,40.0,60.0,75,390,80.0,400.0
3,8086147,739,739,688,688,22.109608,85.548038,68.313953,261.991279,-1,55,40.0,60.0,75,390,80.0,400.0
4,8049498,156,156,155,155,7.339744,76.987179,73.677419,239.354839,1,55,60.0,100.0,19,105,80.0,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,202,202,195,195,-1.000000,64.579208,66.717949,240.512821,-1,30,50.0,100.0,-1,115,80.0,400.0
784709,47006229,202,202,195,195,-1.000000,64.579208,66.717949,240.512821,-1,30,50.0,100.0,-1,115,80.0,400.0
784710,47006331,202,202,195,195,-1.000000,64.579208,66.717949,240.512821,-1,30,50.0,100.0,-1,115,80.0,400.0
784711,47006332,202,202,195,195,-1.000000,64.579208,66.717949,240.512821,-1,30,50.0,100.0,-1,115,80.0,400.0


## CatBoostEncoder

In [ ]:
df = Read()
df = df.drop("建築年", axis=1)
Fvalue_torihiki = pd.read_csv(path_start/"feature_value/df_取引時点.csv")
df = pd.merge(df, Fvalue_torihiki, on="ID")
display(df.head())
print(df.shape)

List_std = list(df.select_dtypes(exclude='object').columns)
List_del = ['ID', '市区町村コード', '取引価格（総額）_log']
for i in List_del:
  List_std.remove(i)

List_Encode = list(df.select_dtypes(include='object').columns)
for c in List_Encode:
  le = LabelEncoder()
  le.fit(df[c])
  df[c] = le.transform(df[c])

# 標準化
scaler = StandardScaler()
scaler.fit(df[List_std])

df[List_std] = scaler.transform(df[List_std])
df.head()

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建物の構造,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時期_年,取引時期_四半期,取引時期_年四半期,建築年,取引-建築
0,8067540,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,70,ＲＣ,...,200.0,2007年第3四半期,未改装,NaN,6.977724,2007,3,2007.50,1996.0,-0.636647
1,8027265,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,70,ＲＣ,...,200.0,2018年第3四半期,未改装,NaN,7.255273,2018,3,2018.50,1998.0,0.106899
2,8061526,8220,茨城県,つくば市,苅間,研究学園,4,３ＬＤＫ,105,ＲＣ,...,400.0,2011年第4四半期,未改装,NaN,7.556303,2011,4,2011.75,2009.0,-1.359538
3,8086147,8220,茨城県,つくば市,並木,つくば,75,４ＬＤＫ,100,ＲＣ,...,200.0,2013年第1四半期,未改装,NaN,7.380211,2013,1,2013.00,2007.0,-1.091036
4,8049498,8224,茨城県,守谷市,ひがし野,守谷,8,４ＬＤＫ,95,ＲＣ,...,200.0,2018年第2四半期,未改装,NaN,7.518514,2018,2,2018.25,2009.0,-0.822533


(784713, 24)


,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建物の構造,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時期_年,取引時期_四半期,取引時期_年四半期,建築年,取引-建築
0,8067540,8217,38,131,11184,2930,0.144049,39,0.431679,4,...,-0.695601,8,1,9,6.977724,-1.603896,0.462829,-1.580274,0.032541,-0.636647
1,8027265,8219,38,449,80,49,-0.266625,39,0.431679,4,...,-0.695601,52,1,9,7.255273,0.754964,0.462829,0.784888,0.201134,0.106899
2,8061526,8220,38,15,9896,2867,-0.595164,39,1.726269,4,...,0.650409,25,1,9,7.556303,-0.746129,1.354318,-0.666462,1.128399,-1.359539
3,8086147,8220,38,15,822,36,5.236408,50,1.541328,4,...,-0.695601,30,1,9,7.380211,-0.317245,-1.320151,-0.397693,0.959805,-1.091036
4,8049498,8224,38,236,79,1360,-0.266625,50,1.356387,4,...,-0.695601,51,1,9,7.518514,0.754964,-0.428661,0.731134,1.128399,-0.822534


In [ ]:
def Func_LoE(df, aim, List_Encode):
  df_a = pd.DataFrame(df["ID"])
  for i in List_Encode:
    loo = ce.CatBoostEncoder(cols=i)
    a = loo.fit_transform(df[i], df[aim])
    a = a.rename(columns={i: '{0}_LoE{1}'.format(i, aim)})
    df_a = pd.concat([df_a, a], axis=1)

  df_a.to_csv(path_start/"feature_value/df_LoE{}.csv".format(aim), index=False)
  display(pd.read_csv(path_start/"feature_value/df_LoE{}.csv".format(aim)))

In [ ]:
for i in List_std:
  Func_LoE(df, i, List_Encode)

,ID,都道府県名_LoE最寄駅：距離（分）,市区町村名_LoE最寄駅：距離（分）,地区名_LoE最寄駅：距離（分）,最寄駅：名称_LoE最寄駅：距離（分）,間取り_LoE最寄駅：距離（分）,建物の構造_LoE最寄駅：距離（分）,用途_LoE最寄駅：距離（分）,今後の利用目的_LoE最寄駅：距離（分）,都市計画_LoE最寄駅：距離（分）,取引時点_LoE最寄駅：距離（分）,改装_LoE最寄駅：距離（分）,取引の事情等_LoE最寄駅：距離（分）
0,8067540,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17,2.274568e-17
1,8027265,7.202451e-02,2.274568e-17,2.274568e-17,2.274568e-17,7.202451e-02,7.202451e-02,7.202451e-02,2.274568e-17,2.274568e-17,2.274568e-17,7.202451e-02,7.202451e-02
2,8061526,-4.085868e-02,2.274568e-17,2.274568e-17,2.274568e-17,-4.085868e-02,-4.085868e-02,-4.085868e-02,7.202451e-02,2.274568e-17,2.274568e-17,-4.085868e-02,-4.085868e-02
3,8086147,-1.794351e-01,-2.975822e-01,2.274568e-17,2.274568e-17,2.274568e-17,-1.794351e-01,-1.794351e-01,-1.503718e-01,2.274568e-17,2.274568e-17,-1.794351e-01,-1.794351e-01
4,8049498,9.037334e-01,2.274568e-17,2.274568e-17,2.274568e-17,2.618204e+00,9.037334e-01,2.274568e-17,-1.333125e-01,2.274568e-17,2.274568e-17,9.037334e-01,9.037334e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-3.767125e-01,-1.000758e+00,-9.579413e-01,-1.005479e+00,-3.990034e-01,4.228669e-02,4.497208e-02,-3.292497e-01,-2.251502e-01,-1.294207e-02,7.764111e-02,-7.084585e-03
784709,47006229,-3.769916e-01,-1.000784e+00,-9.284662e-01,-1.005479e+00,2.812004e-01,4.228472e-02,4.496000e-02,1.102668e-02,1.446837e-01,-1.259486e-03,-1.716346e-02,-7.085890e-03
784710,47006331,-3.772704e-01,-1.000809e+00,-9.143985e-01,-1.005479e+00,-3.011077e-02,4.228274e-02,4.494791e-02,1.102405e-02,1.446741e-01,-1.412716e-03,-8.615794e-02,-7.087195e-03
784711,47006332,-3.775491e-01,-1.000834e+00,-9.220185e-01,-1.005479e+00,-3.011928e-02,3.204326e-02,-5.768270e-04,1.102141e-02,1.446644e-01,-1.301999e-02,-1.716540e-02,-7.088500e-03


,ID,都道府県名_LoE面積（㎡）,市区町村名_LoE面積（㎡）,地区名_LoE面積（㎡）,最寄駅：名称_LoE面積（㎡）,間取り_LoE面積（㎡）,建物の構造_LoE面積（㎡）,用途_LoE面積（㎡）,今後の利用目的_LoE面積（㎡）,都市計画_LoE面積（㎡）,取引時点_LoE面積（㎡）,改装_LoE面積（㎡）,取引の事情等_LoE面積（㎡）
0,8067540,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17,9.634318e-17
1,8027265,2.158396e-01,9.634318e-17,9.634318e-17,9.634318e-17,2.158396e-01,2.158396e-01,2.158396e-01,9.634318e-17,9.634318e-17,9.634318e-17,2.158396e-01,2.158396e-01
2,8061526,2.877862e-01,9.634318e-17,9.634318e-17,9.634318e-17,2.877862e-01,2.877862e-01,2.877862e-01,2.158396e-01,9.634318e-17,9.634318e-17,2.877862e-01,2.877862e-01
3,8086147,6.474070e-01,8.631347e-01,9.634318e-17,9.634318e-17,9.634318e-17,6.474070e-01,6.474070e-01,7.193162e-01,9.634318e-17,9.634318e-17,6.474070e-01,6.474070e-01
4,8049498,8.261912e-01,9.634318e-17,9.634318e-17,9.634318e-17,7.706640e-01,8.261912e-01,9.634318e-17,2.158396e-01,9.634318e-17,9.634318e-17,8.261912e-01,8.261912e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,3.307672e-01,2.212064e-01,1.821480e-01,2.237634e-01,9.167927e-01,6.483499e-03,1.560814e-02,9.931331e-01,-1.986122e-01,-1.125408e-01,6.869695e-02,-2.206119e-03
784709,47006229,3.310581e-01,2.250521e-01,5.549621e-01,2.240361e-01,1.045355e+00,6.485343e-03,1.561931e-02,8.765448e-03,1.531647e-01,-1.233238e-01,8.736516e-03,-2.204827e-03
784710,47006331,3.313488e-01,2.288594e-01,2.579328e-01,2.243086e-01,7.676828e-02,6.487186e-03,1.563048e-02,8.767979e-03,1.531717e-01,-1.231545e-01,-2.338749e-01,-2.203535e-03
784711,47006332,3.314752e-01,2.307885e-01,2.878235e-01,2.244487e-01,7.677299e-02,9.628560e-02,-6.238946e-03,8.769552e-03,1.531756e-01,-1.124546e-01,8.738437e-03,-2.202727e-03


,ID,都道府県名_LoE建ぺい率（％）,市区町村名_LoE建ぺい率（％）,地区名_LoE建ぺい率（％）,最寄駅：名称_LoE建ぺい率（％）,間取り_LoE建ぺい率（％）,建物の構造_LoE建ぺい率（％）,用途_LoE建ぺい率（％）,今後の利用目的_LoE建ぺい率（％）,都市計画_LoE建ぺい率（％）,取引時点_LoE建ぺい率（％）,改装_LoE建ぺい率（％）,取引の事情等_LoE建ぺい率（％）
0,8067540,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16,-5.871815e-16
1,8027265,-3.708648e-01,-5.871815e-16,-5.871815e-16,-5.871815e-16,-3.708648e-01,-3.708648e-01,-3.708648e-01,-5.871815e-16,-5.871815e-16,-5.871815e-16,-3.708648e-01,-3.708648e-01
2,8061526,-4.944864e-01,-5.871815e-16,-5.871815e-16,-5.871815e-16,-4.944864e-01,-4.944864e-01,-4.944864e-01,-3.708648e-01,-5.871815e-16,-5.871815e-16,-4.944864e-01,-4.944864e-01
3,8086147,-7.645911e-02,5.888114e-01,-5.871815e-16,-5.871815e-16,-5.871815e-16,-7.645911e-02,-7.645911e-02,1.452977e-01,-5.871815e-16,-5.871815e-16,-7.645911e-02,-7.645911e-02
4,8049498,-2.095132e-01,-5.871815e-16,-5.871815e-16,-5.871815e-16,-3.708648e-01,-2.095132e-01,-5.871815e-16,-3.708648e-01,-5.871815e-16,-5.871815e-16,-2.095132e-01,-2.095132e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,3.263300e-02,-8.651716e-02,8.473523e-01,-9.924378e-02,8.352935e-01,-1.685245e-01,2.131244e-02,7.987613e-01,1.159028e+00,8.165221e-02,-1.030407e-01,4.131016e-03
784709,47006229,3.314098e-02,-8.016470e-02,-5.381252e-02,-9.878727e-02,-3.218964e-01,-1.685220e-01,2.132574e-02,5.132905e-04,-6.214622e-01,1.071614e-01,1.653125e-02,4.132549e-03
784710,47006331,3.279736e-02,-8.347252e-02,1.981332e-01,-9.901706e-02,2.156141e-02,-1.685231e-01,2.131696e-02,5.113694e-04,-6.214632e-01,1.070319e-01,1.555944e-01,4.131575e-03
784711,47006332,3.245404e-02,-8.674744e-02,1.198113e-01,-9.924668e-02,2.155475e-02,-2.724429e-02,-1.261034e-02,5.094482e-04,-6.214643e-01,8.173821e-02,1.652976e-02,4.130600e-03


,ID,都道府県名_LoE容積率（％）,市区町村名_LoE容積率（％）,地区名_LoE容積率（％）,最寄駅：名称_LoE容積率（％）,間取り_LoE容積率（％）,建物の構造_LoE容積率（％）,用途_LoE容積率（％）,今後の利用目的_LoE容積率（％）,都市計画_LoE容積率（％）,取引時点_LoE容積率（％）,改装_LoE容積率（％）,取引の事情等_LoE容積率（％）
0,8067540,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16,1.795081e-16
1,8027265,-3.478005e-01,1.795081e-16,1.795081e-16,1.795081e-16,-3.478005e-01,-3.478005e-01,-3.478005e-01,1.795081e-16,1.795081e-16,1.795081e-16,-3.478005e-01,-3.478005e-01
2,8061526,-4.637341e-01,1.795081e-16,1.795081e-16,1.795081e-16,-4.637341e-01,-4.637341e-01,-4.637341e-01,-3.478005e-01,1.795081e-16,1.795081e-16,-4.637341e-01,-4.637341e-01
3,8086147,-1.851982e-01,3.252047e-01,1.795081e-16,1.795081e-16,1.795081e-16,-1.851982e-01,-1.851982e-01,-1.506388e-02,1.795081e-16,1.795081e-16,-1.851982e-01,-1.851982e-01
4,8049498,-2.872788e-01,1.795081e-16,1.795081e-16,1.795081e-16,-3.478005e-01,-2.872788e-01,1.795081e-16,-3.478005e-01,1.795081e-16,1.795081e-16,-2.872788e-01,-2.872788e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-1.942817e-01,-4.023727e-01,-1.497113e-01,-1.664395e-01,1.186784e+00,-1.798913e-01,-7.641667e-03,9.867536e-01,2.664945e-02,8.789015e-02,-1.184122e-01,5.197361e-03
784709,47006229,-1.942056e-01,-4.004643e-01,-5.350778e-01,-1.663881e-01,-4.130011e-01,-1.798910e-01,-7.641839e-03,-8.372392e-03,-5.880326e-01,1.134280e-01,2.218287e-02,5.197325e-03
784710,47006331,-1.944279e-01,-4.019400e-01,-2.652708e-01,-1.665772e-01,5.619016e-02,-1.798920e-01,-7.649754e-03,-8.374171e-03,-5.880335e-01,1.133046e-01,1.578369e-01,5.196409e-03
784711,47006332,-1.946501e-01,-4.034010e-01,-3.011317e-01,-1.667662e-01,5.618360e-02,-9.467116e-02,-1.207709e-02,-8.375949e-03,-5.880344e-01,8.788148e-02,2.218146e-02,5.195493e-03


,ID,都道府県名_LoE取引時期_年,市区町村名_LoE取引時期_年,地区名_LoE取引時期_年,最寄駅：名称_LoE取引時期_年,間取り_LoE取引時期_年,建物の構造_LoE取引時期_年,用途_LoE取引時期_年,今後の利用目的_LoE取引時期_年,都市計画_LoE取引時期_年,取引時点_LoE取引時期_年,改装_LoE取引時期_年,取引の事情等_LoE取引時期_年
0,8067540,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16,4.079735e-16
1,8027265,-8.019482e-01,4.079735e-16,4.079735e-16,4.079735e-16,-8.019482e-01,-8.019482e-01,-8.019482e-01,4.079735e-16,4.079735e-16,4.079735e-16,-8.019482e-01,-8.019482e-01
2,8061526,-2.829774e-01,4.079735e-16,4.079735e-16,4.079735e-16,-2.829774e-01,-2.829774e-01,-2.829774e-01,-8.019482e-01,4.079735e-16,4.079735e-16,-2.829774e-01,-2.829774e-01
3,8086147,-3.987653e-01,-3.730644e-01,4.079735e-16,4.079735e-16,4.079735e-16,-3.987653e-01,-3.987653e-01,-7.833417e-01,4.079735e-16,4.079735e-16,-3.987653e-01,-3.987653e-01
4,8049498,-3.824613e-01,4.079735e-16,4.079735e-16,4.079735e-16,-1.586226e-01,-3.824613e-01,4.079735e-16,3.774821e-01,4.079735e-16,4.079735e-16,-3.824613e-01,-3.824613e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,1.364311e-01,2.659494e-01,4.228795e-01,-8.719502e-01,-1.995465e-01,1.496147e-02,1.053829e+00,7.142524e-01,4.530868e-02,1.612605e+00,-5.682914e-03,1.423939e-02
784709,47006229,1.370861e-01,2.727171e-01,1.855286e-01,-8.710618e-01,-1.035613e-01,1.496448e-02,1.053835e+00,7.618092e-01,-1.155297e-02,1.612486e+00,-4.240789e-02,1.424148e-02
784710,47006331,1.377405e-01,2.794172e-01,2.769420e-01,-8.701741e-01,5.518324e-02,1.496748e-02,1.053842e+00,7.618114e-01,-1.153933e-02,1.612486e+00,2.937640e-01,1.424357e-02
784711,47006332,1.383943e-01,2.860506e-01,3.882578e-01,-8.692871e-01,5.519683e-02,4.711033e-01,-1.293057e-01,7.618136e-01,-1.152569e-02,1.612605e+00,-4.240464e-02,1.424566e-02


,ID,都道府県名_LoE取引時期_四半期,市区町村名_LoE取引時期_四半期,地区名_LoE取引時期_四半期,最寄駅：名称_LoE取引時期_四半期,間取り_LoE取引時期_四半期,建物の構造_LoE取引時期_四半期,用途_LoE取引時期_四半期,今後の利用目的_LoE取引時期_四半期,都市計画_LoE取引時期_四半期,取引時点_LoE取引時期_四半期,改装_LoE取引時期_四半期,取引の事情等_LoE取引時期_四半期
0,8067540,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16,-1.382126e-16
1,8027265,2.314143e-01,-1.382126e-16,-1.382126e-16,-1.382126e-16,2.314143e-01,2.314143e-01,2.314143e-01,-1.382126e-16,-1.382126e-16,-1.382126e-16,2.314143e-01,2.314143e-01
2,8061526,3.085523e-01,-1.382126e-16,-1.382126e-16,-1.382126e-16,3.085523e-01,3.085523e-01,3.085523e-01,2.314143e-01,-1.382126e-16,-1.382126e-16,3.085523e-01,3.085523e-01
3,8086147,5.699938e-01,6.771591e-01,-1.382126e-16,-1.382126e-16,-1.382126e-16,5.699938e-01,5.699938e-01,6.057156e-01,-1.382126e-16,-1.382126e-16,5.699938e-01,5.699938e-01
4,8049498,1.919649e-01,-1.382126e-16,-1.382126e-16,-1.382126e-16,-6.600754e-01,1.919649e-01,-1.382126e-16,2.314143e-01,-1.382126e-16,-1.382126e-16,1.919649e-01,1.919649e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,2.934248e-02,1.813377e-01,2.285296e-01,4.625184e-02,1.700635e-02,-4.576958e-03,1.745784e-02,-1.683337e-02,2.454286e-03,-4.286275e-01,7.823540e-03,-7.067035e-04
784709,47006229,2.913928e-02,1.782723e-01,3.586502e-01,4.608205e-02,-3.990649e-03,-4.577756e-03,1.745271e-02,-8.573163e-03,-2.934139e-03,4.627579e-01,-3.515137e-03,-7.072627e-04
784710,47006331,2.933160e-02,1.796951e-01,2.586642e-01,4.623100e-02,-6.075544e-03,-4.576876e-03,1.745783e-02,-8.571943e-03,-2.930228e-03,4.627579e-01,3.076070e-03,-7.066570e-04
784711,47006332,2.952376e-02,1.811037e-01,2.756779e-01,4.637983e-02,-6.071451e-03,1.283339e-02,-2.621320e-03,-8.570723e-03,-2.926317e-03,-4.286275e-01,-3.514221e-03,-7.060513e-04


,ID,都道府県名_LoE取引時期_年四半期,市区町村名_LoE取引時期_年四半期,地区名_LoE取引時期_年四半期,最寄駅：名称_LoE取引時期_年四半期,間取り_LoE取引時期_年四半期,建物の構造_LoE取引時期_年四半期,用途_LoE取引時期_年四半期,今後の利用目的_LoE取引時期_年四半期,都市計画_LoE取引時期_年四半期,取引時点_LoE取引時期_年四半期,改装_LoE取引時期_年四半期,取引の事情等_LoE取引時期_年四半期
0,8067540,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14,2.008574e-14
1,8027265,-7.901372e-01,2.008574e-14,2.008574e-14,2.008574e-14,-7.901372e-01,-7.901372e-01,-7.901372e-01,2.008574e-14,2.008574e-14,2.008574e-14,-7.901372e-01,-7.901372e-01
2,8061526,-2.651288e-01,2.008574e-14,2.008574e-14,2.008574e-14,-2.651288e-01,-2.651288e-01,-2.651288e-01,-7.901372e-01,2.008574e-14,2.008574e-14,-2.651288e-01,-2.651288e-01
3,8086147,-3.654620e-01,-3.332308e-01,2.008574e-14,2.008574e-14,2.008574e-14,-3.654620e-01,-3.654620e-01,-7.489121e-01,2.008574e-14,2.008574e-14,-3.654620e-01,-3.654620e-01
4,8049498,-3.719083e-01,2.008574e-14,2.008574e-14,2.008574e-14,-1.988466e-01,-3.719083e-01,2.008574e-14,3.924440e-01,2.008574e-14,2.008574e-14,-3.719083e-01,-3.719083e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,1.385649e-01,2.775939e-01,4.377888e-01,-8.714909e-01,-1.990542e-01,1.472547e-02,1.057697e+00,7.151457e-01,4.557771e-02,1.591069e+00,-5.226365e-03,1.423482e-02
784709,47006229,1.392093e-01,2.841949e-01,2.076496e-01,-8.706104e-01,-1.040786e-01,1.472843e-02,1.057703e+00,7.633276e-01,-1.176076e-02,1.644696e+00,-4.273314e-02,1.423688e-02
784710,47006331,1.398771e-01,2.909987e-01,2.932785e-01,-8.697114e-01,5.496434e-02,1.473150e-02,1.057710e+00,7.633299e-01,-1.174684e-02,1.644696e+00,2.947343e-01,1.423901e-02
784711,47006332,1.405442e-01,2.977348e-01,4.059175e-01,-8.688130e-01,5.497821e-02,4.731357e-01,-1.298092e-01,7.633322e-01,-1.173293e-02,1.591069e+00,-4.272983e-02,1.424114e-02


,ID,都道府県名_LoE建築年,市区町村名_LoE建築年,地区名_LoE建築年,最寄駅：名称_LoE建築年,間取り_LoE建築年,建物の構造_LoE建築年,用途_LoE建築年,今後の利用目的_LoE建築年,都市計画_LoE建築年,取引時点_LoE建築年,改装_LoE建築年,取引の事情等_LoE建築年
0,8067540,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15,-9.351263e-15
1,8027265,1.627048e-02,-9.351263e-15,-9.351263e-15,-9.351263e-15,1.627048e-02,1.627048e-02,1.627048e-02,-9.351263e-15,-9.351263e-15,-9.351263e-15,1.627048e-02,1.627048e-02
2,8061526,7.789181e-02,-9.351263e-15,-9.351263e-15,-9.351263e-15,7.789181e-02,7.789181e-02,7.789181e-02,1.627048e-02,-9.351263e-15,-9.351263e-15,7.789181e-02,7.789181e-02
3,8086147,3.405186e-01,5.641994e-01,-9.351263e-15,-9.351263e-15,-9.351263e-15,3.405186e-01,3.405186e-01,3.869799e-01,-9.351263e-15,-9.351263e-15,3.405186e-01,3.405186e-01
4,8049498,4.643759e-01,-9.351263e-15,-9.351263e-15,-9.351263e-15,4.799027e-01,4.643759e-01,-9.351263e-15,1.005672e-01,-9.351263e-15,-9.351263e-15,4.643759e-01,4.643759e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,1.982296e-01,2.538888e-01,5.809274e-01,-9.404561e-03,-7.313294e-01,1.936715e-01,1.207565e-01,-5.757788e-01,1.437766e-01,2.268857e-01,-3.579207e-01,3.713507e-03
784709,47006229,1.989414e-01,2.616722e-01,7.887086e-01,-8.756660e-03,1.140445e-01,1.936745e-01,1.207759e-01,1.202892e-01,-1.266208e-01,1.732375e-01,1.149689e-01,3.715858e-03
784710,47006331,1.996153e-01,2.689562e-01,7.163252e-01,-8.139351e-03,-9.892013e-02,1.936774e-01,1.207942e-01,1.202933e-01,-1.266053e-01,1.734732e-01,1.613115e-01,3.718098e-03
784711,47006332,2.003633e-01,2.770065e-01,8.138872e-01,-7.462249e-03,-9.890280e-02,-1.162416e-02,-7.359448e-03,1.202979e-01,-1.265884e-01,2.270093e-01,1.149720e-01,3.720559e-03


,ID,都道府県名_LoE取引-建築,市区町村名_LoE取引-建築,地区名_LoE取引-建築,最寄駅：名称_LoE取引-建築,間取り_LoE取引-建築,建物の構造_LoE取引-建築,用途_LoE取引-建築,今後の利用目的_LoE取引-建築,都市計画_LoE取引-建築,取引時点_LoE取引-建築,改装_LoE取引-建築,取引の事情等_LoE取引-建築
0,8067540,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17,4.357893e-17
1,8027265,-3.183235e-01,4.357893e-17,4.357893e-17,4.357893e-17,-3.183235e-01,-3.183235e-01,-3.183235e-01,4.357893e-17,4.357893e-17,4.357893e-17,-3.183235e-01,-3.183235e-01
2,8061526,-1.765825e-01,4.357893e-17,4.357893e-17,4.357893e-17,-1.765825e-01,-1.765825e-01,-1.765825e-01,-3.183235e-01,4.357893e-17,4.357893e-17,-1.765825e-01,-1.765825e-01
3,8086147,-4.723217e-01,-6.797696e-01,4.357893e-17,4.357893e-17,4.357893e-17,-4.723217e-01,-4.723217e-01,-6.653954e-01,4.357893e-17,4.357893e-17,-4.723217e-01,-4.723217e-01
4,8049498,-5.960646e-01,4.357893e-17,4.357893e-17,4.357893e-17,-5.455182e-01,-5.960646e-01,4.357893e-17,5.344971e-02,4.357893e-17,4.357893e-17,-5.960646e-01,-5.960646e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-1.412990e-01,-1.375493e-01,-3.988062e-01,-2.028956e-01,6.416609e-01,-1.820533e-01,2.607308e-01,8.308711e-01,-1.235533e-01,3.729751e-01,3.487290e-01,1.685782e-03
784709,47006229,-1.417478e-01,-1.426521e-01,-6.909444e-01,-2.032352e-01,-1.525111e-01,-1.820552e-01,2.607145e-01,1.678985e-01,1.189480e-01,4.445720e-01,-1.287064e-01,1.684274e-03
784710,47006331,-1.421505e-01,-1.471875e-01,-6.128974e-01,-2.035378e-01,1.173369e-01,-1.820568e-01,2.606995e-01,1.678954e-01,1.189382e-01,4.443440e-01,-4.729375e-02,1.682900e-03
784711,47006332,-1.426260e-01,-1.524998e-01,-6.630694e-01,-2.038991e-01,1.173252e-01,1.122007e-01,-3.928297e-02,1.678918e-01,1.189270e-01,3.728554e-01,-1.287082e-01,1.681310e-03


In [ ]:
df_r5 = pd.read_csv(path_start/"feature_value/df_国土数値.csv")
df = pd.merge(df, df_r5, on="ID")
Func_LoE(df, "Ｒ５価格", List_Encode)

,ID,Ｒ５価格
0,8067540,-0.164178
1,8027265,-0.169847
2,8061526,-0.095938
3,8086147,-0.095938
4,8049498,0.019509


In [ ]:
df_r5 = pd.read_csv(path_start/"feature_value/df_市区町村人口.csv")
display(df_r5.columns[1])
display(df_r5.head())
df = pd.merge(df, df_r5, on="ID")
Func_LoE(df, df_r5.columns[1], List_Encode)

'市町村別人口'

,ID,市町村別人口
0,8067540,0.389892
1,8072555,0.389892
2,8024138,0.389892
3,8067758,0.389892
4,8045321,0.389892


,ID,都道府県名_LoE市町村別人口,市区町村名_LoE市町村別人口,地区名_LoE市町村別人口,最寄駅：名称_LoE市町村別人口,間取り_LoE市町村別人口,建物の構造_LoE市町村別人口,用途_LoE市町村別人口,今後の利用目的_LoE市町村別人口,都市計画_LoE市町村別人口,取引時点_LoE市町村別人口,改装_LoE市町村別人口,取引の事情等_LoE市町村別人口
0,8067540,2.291169,2.291169,2.291169,2.291169,2.291169,2.291169,2.291169,2.291169,2.291169,2.291169,2.291169,2.291169
1,8027265,1.340531,2.291169,2.291169,2.291169,1.340531,1.340531,1.340531,2.291169,2.291169,2.291169,1.340531,1.340531
2,8061526,0.956345,2.291169,2.291169,2.291169,0.956345,0.956345,0.956345,1.340531,2.291169,2.291169,0.956345,0.956345
3,8086147,1.167518,2.046103,2.291169,2.291169,2.291169,1.167518,1.167518,1.494033,2.291169,2.291169,1.167518,1.167518
4,8049498,1.294222,2.291169,2.291169,2.291169,2.046103,1.294222,2.291169,1.239572,2.291169,2.291169,1.294222,1.294222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,2.067628,0.781515,0.846105,1.959325,1.959249,2.361209,2.191604,1.996302,2.677320,2.261281,2.290476,2.294544
784709,47006229,2.067054,0.781477,0.890569,1.958901,1.903298,2.361206,2.191588,2.300055,2.274989,2.260655,2.292298,2.294542
784710,47006331,2.066480,0.781439,0.911790,1.958477,2.384235,2.361203,2.191571,2.300052,2.274977,2.260428,2.285608,2.294540
784711,47006332,2.065907,0.781401,0.900295,1.958054,2.384221,1.935451,2.307485,2.300048,2.274964,2.261164,2.292295,2.294538


In [ ]:
df_r5 = pd.read_csv(path_start/"feature_value/df_都道府県別人口.csv")
display(df_r5.columns[1])
display(df_r5.head())
df = pd.merge(df, df_r5, on="ID")
Func_LoE(df, df_r5.columns[1], List_Encode)

'都道府県別人口'

,ID,都道府県別人口
0,8067540,0.074012
1,8027265,0.074012
2,8061526,0.074012
3,8086147,0.074012
4,8049498,0.074012


,ID,都道府県名_LoE都道府県別人口,市区町村名_LoE都道府県別人口,地区名_LoE都道府県別人口,最寄駅：名称_LoE都道府県別人口,間取り_LoE都道府県別人口,建物の構造_LoE都道府県別人口,用途_LoE都道府県別人口,今後の利用目的_LoE都道府県別人口,都市計画_LoE都道府県別人口,取引時点_LoE都道府県別人口,改装_LoE都道府県別人口,取引の事情等_LoE都道府県別人口
0,8067540,2.158193,2.158193,2.158193,2.158193,2.158193,2.158193,2.158193,2.158193,2.158193,2.158193,2.158193,2.158193
1,8027265,1.116102,2.158193,2.158193,2.158193,1.116102,1.116102,1.116102,2.158193,2.158193,2.158193,1.116102,1.116102
2,8061526,0.768739,2.158193,2.158193,2.158193,0.768739,0.768739,0.768739,1.116102,2.158193,2.158193,0.768739,0.768739
3,8086147,0.595057,1.116102,2.158193,2.158193,2.158193,0.595057,0.595057,0.768739,2.158193,2.158193,0.595057,0.595057
4,8049498,0.490848,2.158193,2.158193,2.158193,1.116102,0.490848,2.158193,1.116102,2.158193,2.158193,0.490848,0.490848
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-0.430139,-0.418210,-0.307980,0.552484,2.264756,2.189395,2.010801,2.125202,2.285283,2.190474,2.120243,2.166592
784709,47006229,-0.430139,-0.418276,-0.232097,0.552133,1.427033,2.189390,2.010772,2.148114,1.861432,2.193129,2.160406,2.166589
784710,47006331,-0.430140,-0.418341,-0.195881,0.551781,2.319733,2.189385,2.010744,2.148107,1.861412,2.192729,2.241182,2.166586
784711,47006332,-0.430140,-0.418405,-0.215498,0.551430,2.319709,1.677779,2.174711,2.148100,1.861393,2.190268,2.160401,2.166582


In [ ]:
df_r5 = pd.read_csv(path_start/"feature_value/df_駅別乗降客数.csv")
df = df.drop("最寄駅：名称", axis=1)
display(list(df_r5.columns[4:]))
df = pd.merge(df, df_r5, on="ID")
display(df[list(df_r5.columns[4:])].head())

['駅別乗降客数_2011',
 '駅別乗降客数_2012',
 '駅別乗降客数_2013',
 '駅別乗降客数_2014',
 '駅別乗降客数_2015',
 '駅別乗降客数_2016',
 '駅別乗降客数_2017',
 '駅別乗降客数_2018',
 '駅別乗降客数_2019',
 '駅別乗降客数_2020',
 '駅別乗降客数_2021',
 '駅別乗降客数_2022']

,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020,駅別乗降客数_2021,駅別乗降客数_2022
0,-0.488452,-0.569343,-0.575882,-0.599380,-0.608130,-0.615370,-0.614987,-0.604885,-0.616000,-0.639725,-0.630697,-0.630688
1,-0.332201,-0.390694,-0.389167,-0.393209,-0.410235,-0.405988,-0.404771,-0.403159,-0.407023,-0.426199,-0.418776,-0.418779
2,-0.360884,-0.410338,-0.394899,-0.405614,-0.403684,-0.400928,-0.395623,-0.392609,-0.396078,-0.433877,-0.416110,-0.416108
3,-0.025468,-0.004236,0.010233,-0.001249,0.000366,0.011256,-0.018188,-0.040911,-0.036291,-0.155142,-0.080423,-0.080421
4,-0.336418,-0.380372,-0.377403,-0.374107,-0.386366,-0.373490,-0.369011,-0.396690,-0.395442,-0.411871,-0.403826,-0.403831


In [ ]:
for i in list(df_r5.columns[4:]):
  Func_LoE(df, i, List_Encode)

,ID,都道府県名_LoE駅別乗降客数_2011,市区町村名_LoE駅別乗降客数_2011,地区名_LoE駅別乗降客数_2011,最寄駅：名称_LoE駅別乗降客数_2011,間取り_LoE駅別乗降客数_2011,建物の構造_LoE駅別乗降客数_2011,用途_LoE駅別乗降客数_2011,今後の利用目的_LoE駅別乗降客数_2011,都市計画_LoE駅別乗降客数_2011,取引時点_LoE駅別乗降客数_2011,改装_LoE駅別乗降客数_2011,取引の事情等_LoE駅別乗降客数_2011
0,8067540,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17,-4.636063e-17
1,8027265,-2.442260e-01,-4.636063e-17,-4.636063e-17,-4.636063e-17,-2.442260e-01,-2.442260e-01,-2.442260e-01,-4.636063e-17,-4.636063e-17,-4.636063e-17,-2.442260e-01,-2.442260e-01
2,8061526,-2.735511e-01,-4.636063e-17,-4.636063e-17,-4.636063e-17,-2.735511e-01,-2.735511e-01,-2.735511e-01,-2.442260e-01,-4.636063e-17,-4.636063e-17,-2.735511e-01,-2.735511e-01
3,8086147,-2.953844e-01,-1.804421e-01,-4.636063e-17,-4.636063e-17,-4.636063e-17,-2.953844e-01,-2.953844e-01,-2.831120e-01,-4.636063e-17,-4.636063e-17,-2.953844e-01,-2.953844e-01
4,8049498,-2.414011e-01,-4.636063e-17,-4.636063e-17,-4.636063e-17,-1.273398e-02,-2.414011e-01,-4.636063e-17,-1.661007e-01,-4.636063e-17,-4.636063e-17,-2.414011e-01,-2.414011e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-4.750867e-01,-5.119031e-01,-4.900015e-01,-5.143176e-01,3.101237e-01,-3.846261e-03,-1.093917e-02,1.963970e-01,-3.906370e-02,-1.871869e-03,-7.538839e-03,2.369966e-03
784709,47006229,-4.751042e-01,-5.119162e-01,-4.749246e-01,-5.143177e-01,-1.981972e-01,-3.847222e-03,-1.094496e-02,-1.029227e-02,-4.727757e-02,-6.444970e-03,3.002605e-03,2.369290e-03
784710,47006331,-4.751217e-01,-5.119291e-01,-4.677287e-01,-5.143177e-01,5.072145e-02,-3.848182e-03,-1.095075e-02,-1.029358e-02,-4.728149e-02,-6.522465e-03,-3.934135e-04,2.368615e-03
784711,47006332,-4.751391e-01,-5.119419e-01,-4.716265e-01,-5.143178e-01,5.071651e-02,-5.207606e-02,-1.504786e-03,-1.029488e-02,-4.728542e-02,-1.912094e-03,3.001589e-03,2.367940e-03


,ID,都道府県名_LoE駅別乗降客数_2012,市区町村名_LoE駅別乗降客数_2012,地区名_LoE駅別乗降客数_2012,最寄駅：名称_LoE駅別乗降客数_2012,間取り_LoE駅別乗降客数_2012,建物の構造_LoE駅別乗降客数_2012,用途_LoE駅別乗降客数_2012,今後の利用目的_LoE駅別乗降客数_2012,都市計画_LoE駅別乗降客数_2012,取引時点_LoE駅別乗降客数_2012,改装_LoE駅別乗降客数_2012,取引の事情等_LoE駅別乗降客数_2012
0,8067540,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17,-6.461513e-17
1,8027265,-2.846716e-01,-6.461513e-17,-6.461513e-17,-6.461513e-17,-2.846716e-01,-2.846716e-01,-2.846716e-01,-6.461513e-17,-6.461513e-17,-6.461513e-17,-2.846716e-01,-2.846716e-01
2,8061526,-3.200126e-01,-6.461513e-17,-6.461513e-17,-6.461513e-17,-3.200126e-01,-3.200126e-01,-3.200126e-01,-2.846716e-01,-6.461513e-17,-6.461513e-17,-3.200126e-01,-3.200126e-01
3,8086147,-3.425938e-01,-2.051688e-01,-6.461513e-17,-6.461513e-17,-6.461513e-17,-3.425938e-01,-3.425938e-01,-3.265603e-01,-6.461513e-17,-6.461513e-17,-3.425938e-01,-3.425938e-01
4,8049498,-2.749223e-01,-6.461513e-17,-6.461513e-17,-6.461513e-17,-2.118133e-03,-2.749223e-01,-6.461513e-17,-1.953472e-01,-6.461513e-17,-6.461513e-17,-2.749223e-01,-2.749223e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.518844e-01,-5.969584e-01,-5.714178e-01,-5.997741e-01,3.631777e-01,-4.814892e-03,-1.347001e-02,2.378217e-01,-6.146047e-02,3.133143e-04,-1.087745e-02,2.932191e-03
784709,47006229,-5.519057e-01,-5.969737e-01,-5.538357e-01,-5.997742e-01,-2.174229e-01,-4.816012e-03,-1.347676e-02,-1.098841e-02,-7.385570e-02,-7.191148e-03,3.516629e-03,2.931403e-03
784710,47006331,-5.519271e-01,-5.969887e-01,-5.454443e-01,-5.997742e-01,5.484463e-02,-4.817132e-03,-1.348351e-02,-1.098994e-02,-7.386011e-02,-7.281568e-03,4.795025e-03,2.930616e-03
784711,47006332,-5.519484e-01,-5.970037e-01,-5.499896e-01,-5.997743e-01,5.483891e-02,-5.674306e-02,-1.811750e-03,-1.099146e-02,-7.386453e-02,2.662096e-04,3.515444e-03,2.929828e-03


,ID,都道府県名_LoE駅別乗降客数_2013,市区町村名_LoE駅別乗降客数_2013,地区名_LoE駅別乗降客数_2013,最寄駅：名称_LoE駅別乗降客数_2013,間取り_LoE駅別乗降客数_2013,建物の構造_LoE駅別乗降客数_2013,用途_LoE駅別乗降客数_2013,今後の利用目的_LoE駅別乗降客数_2013,都市計画_LoE駅別乗降客数_2013,取引時点_LoE駅別乗降客数_2013,改装_LoE駅別乗降客数_2013,取引の事情等_LoE駅別乗降客数_2013
0,8067540,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18,-8.692618e-18
1,8027265,-2.879409e-01,-8.692618e-18,-8.692618e-18,-8.692618e-18,-2.879409e-01,-2.879409e-01,-2.879409e-01,-8.692618e-18,-8.692618e-18,-8.692618e-18,-2.879409e-01,-2.879409e-01
2,8061526,-3.216830e-01,-8.692618e-18,-8.692618e-18,-8.692618e-18,-3.216830e-01,-3.216830e-01,-3.216830e-01,-2.879409e-01,-8.692618e-18,-8.692618e-18,-3.216830e-01,-3.216830e-01
3,8086147,-3.399871e-01,-1.974496e-01,-8.692618e-18,-8.692618e-18,-8.692618e-18,-3.399871e-01,-3.399871e-01,-3.235937e-01,-8.692618e-18,-8.692618e-18,-3.399871e-01,-3.399871e-01
4,8049498,-2.699431e-01,-8.692618e-18,-8.692618e-18,-8.692618e-18,5.116378e-03,-2.699431e-01,-8.692618e-18,-1.945836e-01,-8.692618e-18,-8.692618e-18,-2.699431e-01,-2.699431e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.527783e-01,-6.023863e-01,-5.766134e-01,-6.052275e-01,3.641219e-01,-4.887706e-03,-1.351438e-02,2.387447e-01,-6.312657e-02,-3.392311e-04,-1.119575e-02,2.971215e-03
784709,47006229,-5.528017e-01,-6.024016e-01,-5.588714e-01,-6.052276e-01,-2.194639e-01,-4.888836e-03,-1.352119e-02,-1.084868e-02,-7.505954e-02,-8.033079e-03,3.751311e-03,2.970420e-03
784710,47006331,-5.528250e-01,-6.024168e-01,-5.504037e-01,-6.052277e-01,5.648848e-02,-4.889966e-03,-1.352800e-02,-1.085022e-02,-7.506400e-02,-8.124204e-03,4.066718e-03,2.969625e-03
784711,47006332,-5.528484e-01,-6.024319e-01,-5.549904e-01,-6.052278e-01,5.648271e-02,-5.588337e-02,-1.794012e-03,-1.085176e-02,-7.506845e-02,-3.867126e-04,3.750115e-03,2.968830e-03


,ID,都道府県名_LoE駅別乗降客数_2014,市区町村名_LoE駅別乗降客数_2014,地区名_LoE駅別乗降客数_2014,最寄駅：名称_LoE駅別乗降客数_2014,間取り_LoE駅別乗降客数_2014,建物の構造_LoE駅別乗降客数_2014,用途_LoE駅別乗降客数_2014,今後の利用目的_LoE駅別乗降客数_2014,都市計画_LoE駅別乗降客数_2014,取引時点_LoE駅別乗降客数_2014,改装_LoE駅別乗降客数_2014,取引の事情等_LoE駅別乗降客数_2014
0,8067540,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17,2.810613e-17
1,8027265,-2.996899e-01,2.810613e-17,2.810613e-17,2.810613e-17,-2.996899e-01,-2.996899e-01,-2.996899e-01,2.810613e-17,2.810613e-17,2.810613e-17,-2.996899e-01,-2.996899e-01
2,8061526,-3.308628e-01,2.810613e-17,2.810613e-17,2.810613e-17,-3.308628e-01,-3.308628e-01,-3.308628e-01,-2.996899e-01,2.810613e-17,2.810613e-17,-3.308628e-01,-3.308628e-01
3,8086147,-3.495506e-01,-2.028070e-01,2.810613e-17,2.810613e-17,2.810613e-17,-3.495506e-01,-3.495506e-01,-3.349980e-01,2.810613e-17,2.810613e-17,-3.495506e-01,-3.495506e-01
4,8049498,-2.798904e-01,2.810613e-17,2.810613e-17,2.810613e-17,-6.246377e-04,-2.798904e-01,2.810613e-17,-1.966043e-01,2.810613e-17,2.810613e-17,-2.798904e-01,-2.798904e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.731328e-01,-6.256803e-01,-5.989108e-01,-6.286315e-01,3.398545e-01,-3.792411e-03,-1.342850e-02,2.091154e-01,-6.367484e-02,1.049224e-03,-1.171532e-02,3.103795e-03
784709,47006229,-5.731575e-01,-6.256963e-01,-5.804828e-01,-6.286315e-01,-2.267156e-01,-3.793587e-03,-1.343558e-02,-9.618828e-03,-7.527493e-02,-1.028470e-02,3.924269e-03,3.102969e-03
784710,47006331,-5.731822e-01,-6.257121e-01,-5.716876e-01,-6.286316e-01,6.180998e-02,-3.794763e-03,-1.344266e-02,-9.620431e-03,-7.527958e-02,-1.037905e-02,4.262620e-03,3.102143e-03
784711,47006332,-5.732069e-01,-6.257277e-01,-5.764517e-01,-6.286317e-01,6.180395e-02,-5.791973e-02,-1.604191e-03,-9.622034e-03,-7.528423e-02,9.997962e-04,3.923026e-03,3.101317e-03


,ID,都道府県名_LoE駅別乗降客数_2015,市区町村名_LoE駅別乗降客数_2015,地区名_LoE駅別乗降客数_2015,最寄駅：名称_LoE駅別乗降客数_2015,間取り_LoE駅別乗降客数_2015,建物の構造_LoE駅別乗降客数_2015,用途_LoE駅別乗降客数_2015,今後の利用目的_LoE駅別乗降客数_2015,都市計画_LoE駅別乗降客数_2015,取引時点_LoE駅別乗降客数_2015,改装_LoE駅別乗降客数_2015,取引の事情等_LoE駅別乗降客数_2015
0,8067540,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.766801e-17
1,8027265,-3.040651e-01,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.040651e-01,-3.040651e-01,-3.040651e-01,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.040651e-01,-3.040651e-01
2,8061526,-3.394550e-01,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.394550e-01,-3.394550e-01,-3.394550e-01,-3.040651e-01,-3.766801e-17,-3.766801e-17,-3.394550e-01,-3.394550e-01
3,8086147,-3.555124e-01,-2.018422e-01,-3.766801e-17,-3.766801e-17,-3.766801e-17,-3.555124e-01,-3.555124e-01,-3.372716e-01,-3.766801e-17,-3.766801e-17,-3.555124e-01,-3.555124e-01
4,8049498,-2.843368e-01,-3.766801e-17,-3.766801e-17,-3.766801e-17,1.827808e-04,-2.843368e-01,-3.766801e-17,-2.051174e-01,-3.766801e-17,-3.766801e-17,-2.843368e-01,-2.843368e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.776209e-01,-6.318890e-01,-6.048539e-01,-6.348694e-01,3.614564e-01,-5.412582e-03,-1.256542e-02,2.158062e-01,-6.949481e-02,-7.709405e-04,-1.412137e-02,3.224368e-03
784709,47006229,-5.776464e-01,-6.319052e-01,-5.862430e-01,-6.348695e-01,-2.327018e-01,-5.413767e-03,-1.257258e-02,-1.086277e-02,-8.286133e-02,-1.266929e-02,4.383488e-03,3.223534e-03
784710,47006331,-5.776719e-01,-6.319211e-01,-5.773605e-01,-6.348696e-01,6.544434e-02,-5.414951e-03,-1.257975e-02,-1.086439e-02,-8.286597e-02,-1.276423e-02,7.413573e-03,3.222700e-03
784711,47006332,-5.776974e-01,-6.319369e-01,-5.821719e-01,-6.348697e-01,6.543822e-02,-5.061851e-02,-2.012485e-03,-1.086600e-02,-8.287061e-02,-8.207149e-04,4.382232e-03,3.221866e-03


...

,ID,都道府県名_LoE駅別乗降客数_2018,市区町村名_LoE駅別乗降客数_2018,地区名_LoE駅別乗降客数_2018,最寄駅：名称_LoE駅別乗降客数_2018,間取り_LoE駅別乗降客数_2018,建物の構造_LoE駅別乗降客数_2018,用途_LoE駅別乗降客数_2018,今後の利用目的_LoE駅別乗降客数_2018,都市計画_LoE駅別乗降客数_2018,取引時点_LoE駅別乗降客数_2018,改装_LoE駅別乗降客数_2018,取引の事情等_LoE駅別乗降客数_2018
0,8067540,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18,5.215571e-18
1,8027265,-3.024426e-01,5.215571e-18,5.215571e-18,5.215571e-18,-3.024426e-01,-3.024426e-01,-3.024426e-01,5.215571e-18,5.215571e-18,5.215571e-18,-3.024426e-01,-3.024426e-01
2,8061526,-3.360149e-01,5.215571e-18,5.215571e-18,5.215571e-18,-3.360149e-01,-3.360149e-01,-3.360149e-01,-3.024426e-01,5.215571e-18,5.215571e-18,-3.360149e-01,-3.360149e-01
3,8086147,-3.501633e-01,-1.963043e-01,5.215571e-18,5.215571e-18,5.215571e-18,-3.501633e-01,-3.501633e-01,-3.324979e-01,5.215571e-18,5.215571e-18,-3.501633e-01,-3.501633e-01
4,8049498,-2.883129e-01,5.215571e-18,5.215571e-18,5.215571e-18,-2.045571e-02,-2.883129e-01,5.215571e-18,-2.015797e-01,5.215571e-18,5.215571e-18,-2.883129e-01,-2.883129e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.671899e-01,-6.249925e-01,-5.982524e-01,-6.279403e-01,4.101725e-01,-1.185580e-02,-1.927529e-02,2.378328e-01,-6.830546e-02,-2.164781e-04,-2.055613e-02,3.096326e-03
784709,47006229,-5.672170e-01,-6.250084e-01,-5.798446e-01,-6.279404e-01,-2.321914e-01,-1.185696e-02,-1.928229e-02,-1.349104e-02,-9.569164e-02,-2.135010e-02,5.178684e-03,3.095501e-03
784710,47006331,-5.672440e-01,-6.250242e-01,-5.710591e-01,-6.279405e-01,5.624904e-02,-1.185812e-02,-1.928930e-02,-1.349263e-02,-9.569611e-02,-2.144266e-02,1.869021e-02,3.094676e-03
784711,47006332,-5.672710e-01,-6.250398e-01,-5.758179e-01,-6.279406e-01,5.624306e-02,-4.409871e-02,-1.642058e-03,-1.349422e-02,-9.570058e-02,-2.657522e-04,5.177439e-03,3.093851e-03


,ID,都道府県名_LoE駅別乗降客数_2019,市区町村名_LoE駅別乗降客数_2019,地区名_LoE駅別乗降客数_2019,最寄駅：名称_LoE駅別乗降客数_2019,間取り_LoE駅別乗降客数_2019,建物の構造_LoE駅別乗降客数_2019,用途_LoE駅別乗降客数_2019,今後の利用目的_LoE駅別乗降客数_2019,都市計画_LoE駅別乗降客数_2019,取引時点_LoE駅別乗降客数_2019,改装_LoE駅別乗降客数_2019,取引の事情等_LoE駅別乗降客数_2019
0,8067540,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17,4.838891e-17
1,8027265,-3.080002e-01,4.838891e-17,4.838891e-17,4.838891e-17,-3.080002e-01,-3.080002e-01,-3.080002e-01,4.838891e-17,4.838891e-17,4.838891e-17,-3.080002e-01,-3.080002e-01
2,8061526,-3.410077e-01,4.838891e-17,4.838891e-17,4.838891e-17,-3.410077e-01,-3.410077e-01,-3.410077e-01,-3.080002e-01,4.838891e-17,4.838891e-17,-3.410077e-01,-3.410077e-01
3,8086147,-3.547753e-01,-1.980392e-01,4.838891e-17,4.838891e-17,4.838891e-17,-3.547753e-01,-3.547753e-01,-3.373596e-01,4.838891e-17,4.838891e-17,-3.547753e-01,-3.547753e-01
4,8049498,-2.910784e-01,4.838891e-17,4.838891e-17,4.838891e-17,-1.814546e-02,-2.910784e-01,4.838891e-17,-2.035113e-01,4.838891e-17,4.838891e-17,-2.910784e-01,-2.910784e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.767704e-01,-6.371814e-01,-6.099198e-01,-6.401867e-01,3.878601e-01,-1.013087e-02,-2.161527e-02,2.307927e-01,-6.528880e-02,2.812081e-03,-1.919322e-02,2.988073e-03
784709,47006229,-5.767987e-01,-6.371976e-01,-5.911530e-01,-6.401868e-01,-2.219087e-01,-1.013206e-02,-2.162238e-02,-1.146085e-02,-9.072257e-02,-1.632657e-02,5.178946e-03,2.987232e-03
784710,47006331,-5.768269e-01,-6.372137e-01,-5.821962e-01,-6.401869e-01,5.241991e-02,-1.013324e-02,-2.162950e-02,-1.146247e-02,-9.072719e-02,-1.642177e-02,1.519152e-02,2.986391e-03
784711,47006332,-5.768551e-01,-6.372296e-01,-5.870478e-01,-6.401870e-01,5.241386e-02,-6.365329e-02,-9.013938e-04,-1.146410e-02,-9.073181e-02,2.761608e-03,5.177678e-03,2.985551e-03


,ID,都道府県名_LoE駅別乗降客数_2020,市区町村名_LoE駅別乗降客数_2020,地区名_LoE駅別乗降客数_2020,最寄駅：名称_LoE駅別乗降客数_2020,間取り_LoE駅別乗降客数_2020,建物の構造_LoE駅別乗降客数_2020,用途_LoE駅別乗降客数_2020,今後の利用目的_LoE駅別乗降客数_2020,都市計画_LoE駅別乗降客数_2020,取引時点_LoE駅別乗降客数_2020,改装_LoE駅別乗降客数_2020,取引の事情等_LoE駅別乗降客数_2020
0,8067540,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17,4.549137e-17
1,8027265,-3.198623e-01,4.549137e-17,4.549137e-17,4.549137e-17,-3.198623e-01,-3.198623e-01,-3.198623e-01,4.549137e-17,4.549137e-17,4.549137e-17,-3.198623e-01,-3.198623e-01
2,8061526,-3.553079e-01,4.549137e-17,4.549137e-17,4.549137e-17,-3.553079e-01,-3.553079e-01,-3.553079e-01,-3.198623e-01,4.549137e-17,4.549137e-17,-3.553079e-01,-3.553079e-01
3,8086147,-3.749501e-01,-2.169383e-01,4.549137e-17,4.549137e-17,4.549137e-17,-3.749501e-01,-3.749501e-01,-3.578671e-01,4.549137e-17,4.549137e-17,-3.749501e-01,-3.749501e-01
4,8049498,-3.309884e-01,4.549137e-17,4.549137e-17,4.549137e-17,-7.757088e-02,-3.309884e-01,4.549137e-17,-2.130996e-01,4.549137e-17,4.549137e-17,-3.309884e-01,-3.309884e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-6.193869e-01,-6.652482e-01,-6.367858e-01,-6.683859e-01,3.653007e-01,-7.156264e-03,-1.896755e-02,2.094677e-01,-6.928121e-02,5.338452e-03,-1.393452e-02,3.007612e-03
784709,47006229,-6.194088e-01,-6.652652e-01,-6.171924e-01,-6.683860e-01,-2.229125e-01,-7.157509e-03,-1.897502e-02,-9.785043e-03,-7.677182e-02,-8.019107e-03,4.120330e-03,3.006734e-03
784710,47006331,-6.194306e-01,-6.652820e-01,-6.078410e-01,-6.683861e-01,5.468307e-02,-7.158753e-03,-1.898250e-02,-9.786748e-03,-7.677679e-02,-8.119870e-03,8.669272e-03,3.005857e-03
784711,47006332,-6.194524e-01,-6.652986e-01,-6.129063e-01,-6.683862e-01,5.467676e-02,-6.306475e-02,-1.184548e-03,-9.788453e-03,-7.678176e-02,5.285568e-03,4.119009e-03,3.004979e-03


,ID,都道府県名_LoE駅別乗降客数_2021,市区町村名_LoE駅別乗降客数_2021,地区名_LoE駅別乗降客数_2021,最寄駅：名称_LoE駅別乗降客数_2021,間取り_LoE駅別乗降客数_2021,建物の構造_LoE駅別乗降客数_2021,用途_LoE駅別乗降客数_2021,今後の利用目的_LoE駅別乗降客数_2021,都市計画_LoE駅別乗降客数_2021,取引時点_LoE駅別乗降客数_2021,改装_LoE駅別乗降客数_2021,取引の事情等_LoE駅別乗降客数_2021
0,8067540,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18,-5.215571e-18
1,8027265,-3.153483e-01,-5.215571e-18,-5.215571e-18,-5.215571e-18,-3.153483e-01,-3.153483e-01,-3.153483e-01,-5.215571e-18,-5.215571e-18,-5.215571e-18,-3.153483e-01,-3.153483e-01
2,8061526,-3.498242e-01,-5.215571e-18,-5.215571e-18,-5.215571e-18,-3.498242e-01,-3.498242e-01,-3.498242e-01,-3.153483e-01,-5.215571e-18,-5.215571e-18,-3.498242e-01,-3.498242e-01
3,8086147,-3.663956e-01,-2.080549e-01,-5.215571e-18,-5.215571e-18,-5.215571e-18,-3.663956e-01,-3.663956e-01,-3.489354e-01,-5.215571e-18,-5.215571e-18,-3.663956e-01,-3.663956e-01
4,8049498,-3.092010e-01,-5.215571e-18,-5.215571e-18,-5.215571e-18,-4.021145e-02,-3.092010e-01,-5.215571e-18,-2.093880e-01,-5.215571e-18,-5.215571e-18,-3.092010e-01,-3.092010e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.995957e-01,-6.540380e-01,-6.260552e-01,-6.571229e-01,3.847710e-01,-8.802374e-03,-1.903341e-02,2.247969e-01,-6.950004e-02,3.052364e-03,-1.672525e-02,3.086205e-03
784709,47006229,-5.996213e-01,-6.540547e-01,-6.067920e-01,-6.571230e-01,-2.282573e-01,-8.803594e-03,-1.904075e-02,-1.121517e-02,-8.585944e-02,-1.325276e-02,4.727616e-03,3.085342e-03
784710,47006331,-5.996469e-01,-6.540712e-01,-5.975982e-01,-6.571230e-01,5.662570e-02,-8.804814e-03,-1.904810e-02,-1.121685e-02,-8.586424e-02,-1.335101e-02,1.183120e-02,3.084479e-03
784711,47006332,-5.996725e-01,-6.540875e-01,-6.025782e-01,-6.571231e-01,5.661947e-02,-5.961935e-02,-1.332792e-03,-1.121852e-02,-8.586903e-02,3.000542e-03,4.726315e-03,3.083616e-03


,ID,都道府県名_LoE駅別乗降客数_2022,市区町村名_LoE駅別乗降客数_2022,地区名_LoE駅別乗降客数_2022,最寄駅：名称_LoE駅別乗降客数_2022,間取り_LoE駅別乗降客数_2022,建物の構造_LoE駅別乗降客数_2022,用途_LoE駅別乗降客数_2022,今後の利用目的_LoE駅別乗降客数_2022,都市計画_LoE駅別乗降客数_2022,取引時点_LoE駅別乗降客数_2022,改装_LoE駅別乗降客数_2022,取引の事情等_LoE駅別乗降客数_2022
0,8067540,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17,7.301799e-17
1,8027265,-3.153441e-01,7.301799e-17,7.301799e-17,7.301799e-17,-3.153441e-01,-3.153441e-01,-3.153441e-01,7.301799e-17,7.301799e-17,7.301799e-17,-3.153441e-01,-3.153441e-01
2,8061526,-3.498224e-01,7.301799e-17,7.301799e-17,7.301799e-17,-3.498224e-01,-3.498224e-01,-3.498224e-01,-3.153441e-01,7.301799e-17,7.301799e-17,-3.498224e-01,-3.498224e-01
3,8086147,-3.663939e-01,-2.080542e-01,7.301799e-17,7.301799e-17,7.301799e-17,-3.663939e-01,-3.663939e-01,-3.489322e-01,7.301799e-17,7.301799e-17,-3.663939e-01,-3.663939e-01
4,8049498,-3.091994e-01,7.301799e-17,7.301799e-17,7.301799e-17,-4.021073e-02,-3.091994e-01,7.301799e-17,-2.093895e-01,7.301799e-17,7.301799e-17,-3.091994e-01,-3.091994e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-5.995934e-01,-6.540377e-01,-6.260550e-01,-6.571226e-01,3.847710e-01,-8.802457e-03,-1.903340e-02,2.247970e-01,-6.950000e-02,3.052370e-03,-1.672530e-02,3.086207e-03
784709,47006229,-5.996190e-01,-6.540544e-01,-6.067917e-01,-6.571227e-01,-2.282572e-01,-8.803677e-03,-1.904074e-02,-1.121515e-02,-8.585943e-02,-1.325277e-02,4.727628e-03,3.085344e-03
784710,47006331,-5.996446e-01,-6.540709e-01,-5.975979e-01,-6.571228e-01,5.662563e-02,-8.804897e-03,-1.904808e-02,-1.121682e-02,-8.586423e-02,-1.335101e-02,1.183127e-02,3.084481e-03
784711,47006332,-5.996702e-01,-6.540873e-01,-6.025779e-01,-6.571229e-01,5.661940e-02,-5.961944e-02,-1.332794e-03,-1.121850e-02,-8.586903e-02,3.000549e-03,4.726328e-03,3.083618e-03


## 仮

In [ ]:
df = Read()
display(df.head())
print(df.shape)

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,8067540,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,70,1996.0,ＲＣ,住宅,NaN,第１種住居地域,60.0,200.0,2007年第3四半期,未改装,NaN,6.977724
1,8027265,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,70,1998.0,ＲＣ,住宅,住宅,第２種中高層住居専用地域,60.0,200.0,2018年第3四半期,未改装,NaN,7.255273
2,8061526,8220,茨城県,つくば市,苅間,研究学園,4,３ＬＤＫ,105,2009.0,ＲＣ,住宅,NaN,商業地域,80.0,400.0,2011年第4四半期,未改装,NaN,7.556303
3,8086147,8220,茨城県,つくば市,並木,つくば,75,４ＬＤＫ,100,2007.0,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2013年第1四半期,未改装,NaN,7.380211
4,8049498,8224,茨城県,守谷市,ひがし野,守谷,8,４ＬＤＫ,95,2009.0,ＲＣ,NaN,住宅,近隣商業地域,80.0,200.0,2018年第2四半期,未改装,NaN,7.518514


(784713, 20)


In [ ]:
List_int = list(df.select_dtypes(exclude='object').columns)
List_int.remove("ID")
List_int.remove("取引価格（総額）_log")
List_int.remove('市区町村コード')
List_int

['最寄駅：距離（分）', '面積（㎡）', '建築年', '建ぺい率（％）', '容積率（％）']

In [ ]:
for i in List_int:
  data = df[i]
  skewness = scipy.stats.skew(data)
  kurtosis = scipy.stats.kurtosis(data, fisher=False)
  print(i)
  print("歪度:", skewness, "尖度:", kurtosis)
  print()

最寄駅：距離（分）
歪度: 3.3275090179544216 尖度: 20.124869992459814

面積（㎡）
歪度: 10.103446991308347 尖度: 553.0912562526349

建築年
歪度: nan 尖度: nan

建ぺい率（％）
歪度: nan 尖度: nan

容積率（％）
歪度: nan 尖度: nan



## 出現頻度

In [ ]:
df = Read()
display(df.head())
print(df.shape)

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,8067540,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,70,1996.0,ＲＣ,住宅,NaN,第１種住居地域,60.0,200.0,2007年第3四半期,未改装,NaN,6.977724
1,8027265,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,70,1998.0,ＲＣ,住宅,住宅,第２種中高層住居専用地域,60.0,200.0,2018年第3四半期,未改装,NaN,7.255273
2,8061526,8220,茨城県,つくば市,苅間,研究学園,4,３ＬＤＫ,105,2009.0,ＲＣ,住宅,NaN,商業地域,80.0,400.0,2011年第4四半期,未改装,NaN,7.556303
3,8086147,8220,茨城県,つくば市,並木,つくば,75,４ＬＤＫ,100,2007.0,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2013年第1四半期,未改装,NaN,7.380211
4,8049498,8224,茨城県,守谷市,ひがし野,守谷,8,４ＬＤＫ,95,2009.0,ＲＣ,NaN,住宅,近隣商業地域,80.0,200.0,2018年第2四半期,未改装,NaN,7.518514


(784713, 20)


In [ ]:
List_categori = list(df.select_dtypes(include='object').columns)
List_categori

['都道府県名',
 '市区町村名',
 '地区名',
 '最寄駅：名称',
 '間取り',
 '建物の構造',
 '用途',
 '今後の利用目的',
 '都市計画',
 '取引時点',
 '改装',
 '取引の事情等']

In [ ]:
df[List_categori] = df[List_categori].fillna("不明")

In [ ]:
List_categori2 = []
for c in List_categori:
  freq = df[c].value_counts()
  df[c] = df[c].map(freq)
  df = df.rename(columns={c:"{}_出現頻度".format(c)})
  List_categori2.append("{}_出現頻度".format(c))
df[List_categori2].head()

,都道府県名_出現頻度,市区町村名_出現頻度,地区名_出現頻度,最寄駅：名称_出現頻度,間取り_出現頻度,建物の構造_出現頻度,用途_出現頻度,今後の利用目的_出現頻度,都市計画_出現頻度,取引時点_出現頻度,改装_出現頻度,取引の事情等_出現頻度
0,3999,690,24,11,329943,531569,689328,368468,119089,8914,509046,765272
1,3999,344,127,237,329943,531569,689328,386351,35564,11996,509046,765272
2,3999,739,26,103,329943,531569,689328,368468,216444,10934,509046,765272
3,3999,739,1265,576,69461,531569,689328,368468,121245,13233,509046,765272
4,3999,156,70,126,69461,531569,86921,386351,79678,12531,509046,765272


In [ ]:
df_frequency = pd.concat([df["ID"], df[List_categori2]], axis=1)
df_frequency.to_csv(path_start/"feature_value/df_出現頻度.csv", index=False)
pd.read_csv(path_start/"feature_value/df_出現頻度.csv")

,ID,都道府県名_出現頻度,市区町村名_出現頻度,地区名_出現頻度,最寄駅：名称_出現頻度,間取り_出現頻度,建物の構造_出現頻度,用途_出現頻度,今後の利用目的_出現頻度,都市計画_出現頻度,取引時点_出現頻度,改装_出現頻度,取引の事情等_出現頻度
0,8067540,3999,690,24,11,329943,531569,689328,368468,119089,8914,509046,765272
1,8027265,3999,344,127,237,329943,531569,689328,386351,35564,11996,509046,765272
2,8061526,3999,739,26,103,329943,531569,689328,368468,216444,10934,509046,765272
3,8086147,3999,739,1265,576,69461,531569,689328,368468,121245,13233,509046,765272
4,8049498,3999,156,70,126,69461,531569,86921,386351,79678,12531,509046,765272
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,2257,202,21,2800,5986,531569,86921,1525,79678,12745,198370,765272
784709,47006229,2257,202,13,2800,69461,531569,86921,386351,119089,6556,509046,765272
784710,47006331,2257,202,12,2800,114580,531569,86921,386351,119089,6556,77297,765272
784711,47006332,2257,202,12,2800,114580,22552,689328,386351,119089,12745,509046,765272


## pca

In [ ]:
df = Read()
df = df.drop("建築年", axis=1)
Fvalue_torihiki = pd.read_csv(path_start/"feature_value/df_取引時点.csv")
df = pd.merge(df, Fvalue_torihiki, on="ID")

List_std = list(df.select_dtypes(exclude='object').columns)
List_del = ['ID', '市区町村コード', '取引価格（総額）_log']
for i in List_del:
  List_std.remove(i)

List_Encode = list(df.select_dtypes(include='object').columns)
for c in List_Encode:
  le = LabelEncoder()
  le.fit(df[c])
  df[c] = le.transform(df[c])

# 標準化
scaler = StandardScaler()
scaler.fit(df[List_std])

df[List_std] = scaler.transform(df[List_std])
df = df.fillna(-1)

pca = PCA(n_components=5)
pca.fit(df)
df2 = pca.transform(df)
df2 = pd.DataFrame(df2)
df2 = df2.rename(columns={0:"pca_1", 1:"pca_2", 2:"pca_3", 3:"pca_4", 4:"pca_5"})
display(df2.shape)
df2.head()

(784713, 5)

,pca_1,pca_2,pca_3,pca_4,pca_5
0,-1.044608e+07,-5403.721298,-565.632224,223.751718,128.508131
1,-1.048635e+07,5885.458879,1451.675270,-177.264155,233.016803
2,-1.045209e+07,-4114.432501,-597.135897,336.992446,118.373051
3,-1.042747e+07,5148.718008,1534.791716,241.136794,104.649805
4,-1.046412e+07,5787.830754,151.758866,69.857509,171.090232


In [ ]:
df_con = pd.concat([df["ID"], df2], axis=1)
df_con.to_csv(path_start/"feature_value/df_PCA.csv", index=False)
pd.read_csv(path_start/"feature_value/df_PCA.csv")

,ID,pca_1,pca_2,pca_3,pca_4,pca_5
0,8067540,-1.044608e+07,-5403.721298,-565.632224,223.751718,128.508131
1,8027265,-1.048635e+07,5885.458879,1451.675270,-177.264155,233.016803
2,8061526,-1.045209e+07,-4114.432501,-597.135897,336.992446,118.373051
3,8086147,-1.042747e+07,5148.718008,1534.791716,241.136794,104.649805
4,8049498,-1.046412e+07,5787.830754,151.758866,69.857509,171.090232
...,...,...,...,...,...,...
784708,47006020,2.849242e+07,-1428.604811,-1755.919143,13.099388,94.674970
784709,47006229,2.849263e+07,1648.474506,-1990.431698,11.091042,91.108427
784710,47006331,2.849273e+07,1184.819612,-1955.097232,11.404666,91.311562
784711,47006332,2.849273e+07,1184.819429,-1955.096982,11.402228,91.098300


## target encoding

In [3]:
df = Read()
df = df.drop("建築年", axis=1)
Fvalue_torihiki = pd.read_csv(path_start/"feature_value/df_取引時点.csv")
df = pd.merge(df, Fvalue_torihiki, on="ID")
display(df.head())
print(df.shape)

List_std = list(df.select_dtypes(exclude='object').columns)
List_del = ['ID', '市区町村コード', '取引価格（総額）_log']
for i in List_del:
  List_std.remove(i)

List_Encode = list(df.select_dtypes(include='object').columns)
for c in List_Encode:
  le = LabelEncoder()
  le.fit(df[c])
  df[c] = le.transform(df[c])

# 標準化
scaler = StandardScaler()
scaler.fit(df[List_std])

df[List_std] = scaler.transform(df[List_std])
df.head()

,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建物の構造,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時期_年,取引時期_四半期,取引時期_年四半期,建築年,取引-建築
0,8067540,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,70,ＲＣ,...,200.0,2007年第3四半期,未改装,NaN,6.977724,2007,3,2007.50,1996.0,-0.636647
1,8027265,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,70,ＲＣ,...,200.0,2018年第3四半期,未改装,NaN,7.255273,2018,3,2018.50,1998.0,0.106899
2,8061526,8220,茨城県,つくば市,苅間,研究学園,4,３ＬＤＫ,105,ＲＣ,...,400.0,2011年第4四半期,未改装,NaN,7.556303,2011,4,2011.75,2009.0,-1.359538
3,8086147,8220,茨城県,つくば市,並木,つくば,75,４ＬＤＫ,100,ＲＣ,...,200.0,2013年第1四半期,未改装,NaN,7.380211,2013,1,2013.00,2007.0,-1.091036
4,8049498,8224,茨城県,守谷市,ひがし野,守谷,8,４ＬＤＫ,95,ＲＣ,...,200.0,2018年第2四半期,未改装,NaN,7.518514,2018,2,2018.25,2009.0,-0.822533


(784713, 24)


,ID,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建物の構造,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時期_年,取引時期_四半期,取引時期_年四半期,建築年,取引-建築
0,8067540,8217,38,131,11184,2930,0.144049,39,0.431679,4,...,-0.695601,8,1,9,6.977724,-1.603896,0.462829,-1.580274,0.032541,-0.636647
1,8027265,8219,38,449,80,49,-0.266625,39,0.431679,4,...,-0.695601,52,1,9,7.255273,0.754964,0.462829,0.784888,0.201134,0.106899
2,8061526,8220,38,15,9896,2867,-0.595164,39,1.726269,4,...,0.650409,25,1,9,7.556303,-0.746129,1.354318,-0.666462,1.128399,-1.359539
3,8086147,8220,38,15,822,36,5.236408,50,1.541328,4,...,-0.695601,30,1,9,7.380211,-0.317245,-1.320151,-0.397693,0.959805,-1.091036
4,8049498,8224,38,236,79,1360,-0.266625,50,1.356387,4,...,-0.695601,51,1,9,7.518514,0.754964,-0.428661,0.731134,1.128399,-0.822534


In [4]:
def Func_LoE(df, aim, List_Encode):
  df_a = pd.DataFrame(df["ID"])
  for i in List_Encode:
    loo = ce.TargetEncoder(cols=i)
    a = loo.fit_transform(df[i], df[aim])
    a = a.rename(columns={i: '{0}_TE{1}'.format(i, aim)})
    df_a = pd.concat([df_a, a], axis=1)

  df_a.to_csv(path_start/"TargetEncoding/df_TE{}.csv".format(aim), index=False)
  display(pd.read_csv(path_start/"TargetEncoding/df_TE{}.csv".format(aim)))

In [5]:
for i in List_std:
  Func_LoE(df, i, List_Encode)

,ID,都道府県名_TE最寄駅：距離（分）,市区町村名_TE最寄駅：距離（分）,地区名_TE最寄駅：距離（分）,最寄駅：名称_TE最寄駅：距離（分）,間取り_TE最寄駅：距離（分）,建物の構造_TE最寄駅：距離（分）,用途_TE最寄駅：距離（分）,今後の利用目的_TE最寄駅：距離（分）,都市計画_TE最寄駅：距離（分）,取引時点_TE最寄駅：距離（分）,改装_TE最寄駅：距離（分）,取引の事情等_TE最寄駅：距離（分）
0,8067540,0.383409,0.065247,-0.051035,-0.051169,0.125702,0.042279,-0.000578,-0.001985,0.144656,0.067857,-0.017167,-0.007091
1,8027265,0.383409,-0.405586,-0.414070,-0.519268,0.125702,0.042279,-0.000578,0.011019,0.330220,-0.000207,-0.017167,-0.007091
2,8061526,0.383409,0.892265,-0.384272,-0.587841,0.125702,0.042279,-0.000578,-0.001985,-0.402811,0.064821,-0.017167,-0.007091
3,8086147,0.383409,0.892265,0.133985,1.064501,0.281186,0.042279,-0.000578,-0.001985,0.425628,0.037503,-0.017167,-0.007091
4,8049498,0.383409,-0.320855,-0.163442,-0.262707,0.281186,0.042279,0.044924,0.011019,-0.225163,-0.013686,-0.017167,-0.007091
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-0.378273,-1.005838,-0.528044,-1.005838,-0.399171,0.042279,0.044924,-0.329909,-0.225163,-0.013177,0.077636,-0.007091
784709,47006229,-0.378273,-1.005838,-0.333749,-1.005838,0.281186,0.042279,0.044924,0.011019,0.144656,-0.001566,-0.017167,-0.007091
784710,47006331,-0.378273,-1.005838,-0.311836,-1.005838,-0.030137,0.042279,0.044924,0.011019,0.144656,-0.001566,-0.086183,-0.007091
784711,47006332,-0.378273,-1.005838,-0.311836,-1.005838,-0.030137,0.031999,-0.000578,0.011019,0.144656,-0.013177,-0.017167,-0.007091


,ID,都道府県名_TE面積（㎡）,市区町村名_TE面積（㎡）,地区名_TE面積（㎡）,最寄駅：名称_TE面積（㎡）,間取り_TE面積（㎡）,建物の構造_TE面積（㎡）,用途_TE面積（㎡）,今後の利用目的_TE面積（㎡）,都市計画_TE面積（㎡）,取引時点_TE面積（㎡）,改装_TE面積（㎡）,取引の事情等_TE面積（㎡）
0,8067540,0.454849,0.201441,0.216920,0.105338,0.432998,0.006488,-0.006239,0.025406,0.153178,-0.006753,0.008739,-0.002202
1,8027265,0.454849,0.506946,0.692329,0.641592,0.432998,0.006488,-0.006239,0.008770,0.295677,-0.019636,0.008739,-0.002202
2,8061526,0.454849,1.006775,1.036502,1.280655,0.432998,0.006488,-0.006239,0.025406,-0.360357,0.038844,0.008739,-0.002202
3,8086147,0.454849,1.006775,0.129633,0.967881,1.045369,0.006488,-0.006239,0.025406,0.306353,0.058135,0.008739,-0.002202
4,8049498,0.454849,0.690122,0.706970,0.736961,1.045369,0.006488,0.015638,0.008770,-0.198600,-0.014300,0.008739,-0.002202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,0.331465,0.231173,0.120285,0.224479,0.916957,0.006488,0.015638,0.993780,-0.198600,-0.112422,0.068702,-0.002202
784709,47006229,0.331465,0.231173,0.209323,0.224479,1.045369,0.006488,0.015638,0.008770,0.153178,-0.123060,0.008739,-0.002202
784710,47006331,0.331465,0.231173,0.095607,0.224479,0.076775,0.006488,0.015638,0.008770,0.153178,-0.123060,-0.233863,-0.002202
784711,47006332,0.331465,0.231173,0.095607,0.224479,0.076775,0.096297,-0.006239,0.008770,0.153178,-0.112422,0.008739,-0.002202


,ID,都道府県名_TE建ぺい率（％）,市区町村名_TE建ぺい率（％）,地区名_TE建ぺい率（％）,最寄駅：名称_TE建ぺい率（％）,間取り_TE建ぺい率（％）,建物の構造_TE建ぺい率（％）,用途_TE建ぺい率（％）,今後の利用目的_TE建ぺい率（％）,都市計画_TE建ぺい率（％）,取引時点_TE建ぺい率（％）,改装_TE建ぺい率（％）,取引の事情等_TE建ぺい率（％）
0,8067540,0.028126,-0.503946,-0.468004,-0.239615,-0.262674,-0.172931,-0.013011,-0.031533,-0.625208,-0.077014,0.017040,0.004265
1,8027265,0.028126,0.488914,0.532706,0.669076,-0.262674,-0.172931,-0.013011,0.000518,-0.751927,0.025454,0.017040,0.004265
2,8061526,0.028126,0.056141,0.569687,0.838683,-0.262674,-0.172931,-0.013011,-0.031533,1.175791,-0.023054,0.017040,0.004265
3,8086147,0.028126,0.056141,-0.031919,-0.044778,-0.335152,-0.172931,-0.013011,-0.031533,-0.782462,-0.036841,0.017040,0.004265
4,8049498,0.028126,0.570859,0.788440,0.548060,-0.335152,-0.172931,0.022271,0.000518,1.163907,0.018804,0.017040,0.004265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,0.035129,-0.097024,0.474282,-0.245841,0.863475,-0.172931,0.022271,0.815578,1.163907,0.083242,-0.106234,0.004265
784709,47006229,0.035129,-0.097024,-0.037873,-0.245841,-0.335152,-0.172931,0.022271,0.000518,-0.625208,0.108558,0.017040,0.004265
784710,47006331,0.035129,-0.097024,0.017982,-0.245841,0.022194,-0.172931,0.022271,0.000518,-0.625208,0.108558,0.164265,0.004265
784711,47006332,0.035129,-0.097024,0.017982,-0.245841,0.022194,-0.037334,-0.013011,0.000518,-0.625208,0.083242,0.017040,0.004265


,ID,都道府県名_TE容積率（％）,市区町村名_TE容積率（％）,地区名_TE容積率（％）,最寄駅：名称_TE容積率（％）,間取り_TE容積率（％）,建物の構造_TE容積率（％）,用途_TE容積率（％）,今後の利用目的_TE容積率（％）,都市計画_TE容積率（％）,取引時点_TE容積率（％）,改装_TE容積率（％）,取引の事情等_TE容積率（％）
0,8067540,-0.213565,-0.576078,-0.433236,-0.218749,-0.319281,-0.184597,-0.012461,-0.030791,-0.591577,-0.102286,0.022867,0.005365
1,8027265,-0.213565,0.113985,0.203528,0.272203,-0.319281,-0.184597,-0.012461,-0.008556,-0.702904,0.028981,0.022867,0.005365
2,8061526,-0.213565,-0.278397,0.306295,0.432554,-0.319281,-0.184597,-0.012461,-0.030791,1.318864,-0.036798,0.022867,0.005365
3,8086147,-0.213565,-0.278397,-0.416577,-0.396203,-0.430004,-0.184597,-0.012461,-0.030791,-0.722616,-0.043977,0.022867,0.005365
4,8049498,-0.213565,-0.430740,-0.690946,-0.361781,-0.430004,-0.184597,-0.008015,-0.008556,0.026761,0.019880,0.022867,0.005365
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-0.215731,-0.422948,-0.079160,-0.412121,1.226531,-0.184597,-0.008015,1.006540,0.026761,0.089514,-0.122088,0.005365
784709,47006229,-0.215731,-0.422948,-0.201064,-0.412121,-0.430004,-0.184597,-0.008015,-0.008556,-0.591577,0.114934,0.022867,0.005365
784710,47006331,-0.215731,-0.422948,-0.111330,-0.412121,0.057872,-0.184597,-0.008015,-0.008556,-0.591577,0.114934,0.166634,0.005365
784711,47006332,-0.215731,-0.422948,-0.111330,-0.412121,0.057872,-0.129618,-0.012461,-0.008556,-0.591577,0.089514,0.022867,0.005365


,ID,都道府県名_TE取引時期_年,市区町村名_TE取引時期_年,地区名_TE取引時期_年,最寄駅：名称_TE取引時期_年,間取り_TE取引時期_年,建物の構造_TE取引時期_年,用途_TE取引時期_年,今後の利用目的_TE取引時期_年,都市計画_TE取引時期_年,取引時点_TE取引時期_年,改装_TE取引時期_年,取引の事情等_TE取引時期_年
0,8067540,-0.004457,-0.078562,0.109631,-0.086065,-0.042201,0.014974,-0.129303,-0.861343,-0.011512,-1.603896,-0.042401,0.01425
1,8027265,-0.004457,0.000677,0.216322,0.045587,-0.042201,0.014974,-0.129303,0.761818,-0.039334,0.754964,-0.042401,0.01425
2,8061526,-0.004457,0.053893,-0.471092,0.459212,-0.042201,0.014974,-0.129303,-0.861343,0.061609,-0.746129,-0.042401,0.01425
3,8086147,-0.004457,0.053893,-0.034657,-0.044725,-0.103538,0.014974,-0.129303,-0.861343,-0.012513,-0.317245,-0.042401,0.01425
4,8049498,-0.004457,0.277968,0.287383,0.417974,-0.103538,0.014974,1.053867,0.761818,0.045329,0.754964,-0.042401,0.01425
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,0.139762,0.300602,0.262320,-0.867825,-0.199277,0.014974,1.053867,0.715310,0.045329,1.612732,-0.005675,0.01425
784709,47006229,0.139762,0.300602,0.102724,-0.867825,-0.103538,0.014974,1.053867,0.761818,-0.011512,1.612732,-0.042401,0.01425
784710,47006331,0.139762,0.300602,0.162035,-0.867825,0.055225,0.014974,1.053867,0.761818,-0.011512,1.612732,0.293802,0.01425
784711,47006332,0.139762,0.300602,0.162035,-0.867825,0.055225,0.471175,-0.129303,0.761818,-0.011512,1.612732,-0.042401,0.01425


,ID,都道府県名_TE取引時期_四半期,市区町村名_TE取引時期_四半期,地区名_TE取引時期_四半期,最寄駅：名称_TE取引時期_四半期,間取り_TE取引時期_四半期,建物の構造_TE取引時期_四半期,用途_TE取引時期_四半期,今後の利用目的_TE取引時期_四半期,都市計画_TE取引時期_四半期,取引時点_TE取引時期_四半期,改装_TE取引時期_四半期,取引の事情等_TE取引時期_四半期
0,8067540,-0.022709,-0.068189,-0.078726,-0.170757,-0.009897,-0.004577,-0.002622,0.007950,-0.002930,0.462829,-0.003515,-0.000707
1,8027265,-0.022709,0.024858,0.027612,0.015203,-0.009897,-0.004577,-0.002622,-0.008572,-0.013849,0.462829,-0.003515,-0.000707
2,8061526,-0.022709,-0.059519,0.254552,0.047365,-0.009897,-0.004577,-0.002622,0.007950,0.003825,1.354318,-0.003515,-0.000707
3,8086147,-0.022709,-0.059519,-0.045285,-0.055659,-0.003984,-0.004577,-0.002622,0.007950,-0.007030,-1.320151,-0.003515,-0.000707
4,8049498,-0.022709,0.045657,-0.033632,0.017083,-0.003984,-0.004577,0.017458,-0.008572,0.002449,-0.428661,-0.003515,-0.000707
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,0.029131,0.175963,0.109257,0.046057,0.016935,-0.004577,0.017458,-0.017114,0.002449,-0.428661,0.007821,-0.000707
784709,47006229,0.029131,0.175963,0.130818,0.046057,-0.003984,-0.004577,0.017458,-0.008572,-0.002930,0.462829,-0.003515,-0.000707
784710,47006331,0.029131,0.175963,0.074393,0.046057,-0.006079,-0.004577,0.017458,-0.008572,-0.002930,0.462829,0.003076,-0.000707
784711,47006332,0.029131,0.175963,0.074393,0.046057,-0.006079,0.012814,-0.002622,-0.008572,-0.002930,-0.428661,-0.003515,-0.000707


,ID,都道府県名_TE取引時期_年四半期,市区町村名_TE取引時期_年四半期,地区名_TE取引時期_年四半期,最寄駅：名称_TE取引時期_年四半期,間取り_TE取引時期_年四半期,建物の構造_TE取引時期_年四半期,用途_TE取引時期_年四半期,今後の利用目的_TE取引時期_年四半期,都市計画_TE取引時期_年四半期,取引時点_TE取引時期_年四半期,改装_TE取引時期_年四半期,取引の事情等_TE取引時期_年四半期
0,8067540,-0.005838,-0.082884,0.105177,-0.096591,-0.042911,0.014738,-0.129807,-0.863165,-0.011720,-1.580274,-0.042727,0.014245
1,8027265,-0.005838,0.002178,0.218565,0.046625,-0.042911,0.014738,-0.129807,0.763336,-0.040274,0.784888,-0.042727,0.014245
2,8061526,-0.005838,0.050448,-0.457002,0.463295,-0.042911,0.014738,-0.129807,-0.863165,0.062004,-0.666462,-0.042727,0.014245
3,8086147,-0.005838,0.050448,-0.037480,-0.048201,-0.104055,0.014738,-0.129807,-0.863165,-0.012970,-0.397693,-0.042727,0.014245
4,8049498,-0.005838,0.281464,0.286122,0.420120,-0.104055,0.014738,1.057735,0.763336,0.045598,0.731134,-0.042727,0.014245
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,0.141892,0.312015,0.269608,-0.867366,-0.198788,0.014738,1.057735,0.716189,0.045598,1.591193,-0.005218,0.014245
784709,47006229,0.141892,0.312015,0.110886,-0.867366,-0.104055,0.014738,1.057735,0.763336,-0.011720,1.644947,-0.042727,0.014245
784710,47006331,0.141892,0.312015,0.166954,-0.867366,0.055006,0.014738,1.057735,0.763336,-0.011720,1.644947,0.294772,0.014245
784711,47006332,0.141892,0.312015,0.166954,-0.867366,0.055006,0.473206,-0.129807,0.763336,-0.011720,1.591193,-0.042727,0.014245


,ID,都道府県名_TE建築年,市区町村名_TE建築年,地区名_TE建築年,最寄駅：名称_TE建築年,間取り_TE建築年,建物の構造_TE建築年,用途_TE建築年,今後の利用目的_TE建築年,都市計画_TE建築年,取引時点_TE建築年,改装_TE建築年,取引の事情等_TE建築年
0,8067540,0.319404,-0.535413,0.018693,0.038202,0.046110,0.198362,-0.007540,-0.125225,-0.130067,-0.285807,0.117855,0.003836
1,8027265,0.319404,0.700533,0.781342,0.808523,0.046110,0.198362,-0.007540,0.124574,-0.145897,0.103857,0.117855,0.003836
2,8061526,0.319404,0.828869,0.728558,1.245702,0.046110,0.198362,-0.007540,-0.125225,0.201008,-0.111762,0.117855,0.003836
3,8086147,0.319404,0.828869,-0.208416,0.790180,0.117632,0.198362,-0.007540,-0.125225,-0.374216,-0.058149,0.117855,0.003836
4,8049498,0.319404,1.041705,1.075009,1.121651,0.117632,0.198362,0.130096,0.124574,0.147599,0.094298,0.117855,0.003836
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,0.213075,0.297103,0.350042,-0.008792,-0.750460,0.198362,0.130096,-0.601827,0.147599,0.234207,-0.367398,0.003836
784709,47006229,0.213075,0.297103,0.305566,-0.008792,0.117632,0.198362,0.130096,0.124574,-0.130067,0.179426,0.117855,0.003836
784710,47006331,0.213075,0.297103,0.264142,-0.008792,-0.101702,0.198362,0.130096,0.124574,-0.130067,0.179426,0.173440,0.003836
784711,47006332,0.213075,0.297103,0.264142,-0.008792,-0.101702,-0.015306,-0.007540,0.124574,-0.130067,0.234207,0.117855,0.003836


,ID,都道府県名_TE取引-建築,市区町村名_TE取引-建築,地区名_TE取引-建築,最寄駅：名称_TE取引-建築,間取り_TE取引-建築,建物の構造_TE取引-建築,用途_TE取引-建築,今後の利用目的_TE取引-建築,都市計画_TE取引-建築,取引時点_TE取引-建築,改装_TE取引-建築,取引の事情等_TE取引-建築
0,8067540,-0.318099,0.495526,0.026846,-0.073848,-0.062626,-0.186458,-0.040248,-0.208311,0.122200,-0.324646,-0.131934,0.001731
1,8027265,-0.318099,-0.731708,-0.704945,-0.802986,-0.062626,-0.186458,-0.040248,0.173860,0.127286,0.202238,-0.131934,0.001731
2,8061526,-0.318099,-0.790001,-0.888053,-1.037553,-0.062626,-0.186458,-0.040248,-0.208311,-0.172099,-0.144102,-0.131934,0.001731
3,8086147,-0.318099,-0.790001,0.196723,-0.789819,-0.157291,-0.186458,-0.040248,-0.208311,0.361626,-0.093375,-0.131934,0.001731
4,8049498,-0.318099,-0.902655,-0.926995,-0.924278,-0.157291,-0.186458,0.280640,0.173860,-0.126833,0.190952,-0.131934,0.001731
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,-0.151383,-0.162239,-0.238189,-0.269796,0.658520,-0.186458,0.280640,0.869454,-0.126833,0.384297,0.357966,0.001731
784709,47006229,-0.151383,-0.162239,-0.256057,-0.269796,-0.157291,-0.186458,0.280640,0.173860,0.122200,0.458640,-0.131934,0.001731
784710,47006331,-0.151383,-0.162239,-0.200305,-0.269796,0.120661,-0.186458,0.280640,0.173860,0.122200,0.458640,-0.050869,0.001731
784711,47006332,-0.151383,-0.162239,-0.200305,-0.269796,0.120661,0.147915,-0.040248,0.173860,0.122200,0.384297,-0.131934,0.001731
